In [1]:
%matplotlib inline

import glob
import itertools
import json
import more_itertools
import nibabel
import nilearn
import numpy as np
import os
import pandas as pd
import re
import scipy
import seaborn as sns
import skimage
import sklearn
import tqdm

from collections import Counter
from load_confounds import Minimal
from matplotlib import pyplot as plt
from nibabel.nifti1 import Nifti1Image
from nilearn import image as nimage
from nilearn import plotting as niplot
from nilearn.glm.first_level import FirstLevelModel, check_events
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.input_data import MultiNiftiMasker, NiftiLabelsMasker
from nilearn.input_data import NiftiMapsMasker, NiftiMasker
from nilearn.input_data import NiftiSpheresMasker
from os import PathLike
from pathlib import Path, PosixPath
from random import sample

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.svm import SVC, LinearSVC


from scipy.spatial import procrustes
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, FastICA
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import make_scorer, recall_score, pairwise_distances
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.utils import Bunch
from tqdm.notebook import tqdm as tqdm_
from typing import Iterable, Sequence, Union

from cimaq_decoding_pipeline import get_fmri_sessions, fetch_fmriprep_session
from cimaq_decoding_pipeline import get_contrasts, get_all_contrasts, get_glm_events

sns.set(rc={'figure.figsize': (15,15)})

from get_difumo import get_difumo
from get_difumo_cut_coords import get_difumo_cut_coords
from scipy.cluster import hierarchy
#libraries need to be installed in conda environment with pip install

/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# help(hierarchy.ward
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import correlation
# help(correlation)
from sklearn.metrics import pairwise
# pairwise.PAIRWISE_DISTANCE_FUNCTIONS
# sorted(dir(scipy.spatial.distance))
from scipy.stats import spearmanr

# pairwise_distances(X=pd.DataFrame([[1,2,[3]]]).values, metric=spearmanr)

In [ ]:
!python3 -m pip install nitime

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import nitime.utils as utils
import nitime.timeseries as ts
import nitime.viz as viz



ModuleNotFoundError: No module named 'nitime'

In [3]:
from multiprocessing import Pool, Process

# help(Process)
import brainiak


def fullfact(dims) -> np.array:
    '''
    Replicates MATLAB's fullfact function (behaves the same way).
    '''
    vals = np.asmatrix(range(1, dims[0] + 1)).T
    if len(dims) == 1:
        return vals
    else:
        aftervals = np.asmatrix(fullfact(dims[1:]))
        inds = np.asmatrix(np.zeros((np.prod(dims), len(dims))))
        row = 0
        for i in range(aftervals.shape[0]):
            inds[row:(row + len(vals)), 0] = vals
            inds[row:(row + len(vals)), 1:] = np.tile(aftervals[i, :],
                                                      (len(vals), 1))
            row += len(vals)
    return inds



In [4]:
os.chdir('/data/simexp/fnadeau/')

atlases_dir = '/data/simexp/fnadeau/nilearn_atlases/difumo_atlases/'
fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/'
events_dir = '/data/simexp/fnadeau/CIMAQ_AS_BIDS_4/'
masker_dir = '/data/simexp/fnadeau/cimaq_maps_maskers/'


from cimaq_decoding_pipeline import get_fmri_sessions
from multiprocessing import Pool, Process
from cimaq_decoding_utils import get_sub_ses_key

v03, v10 = [get_fmri_sessions(topdir=fmriprep_dir,
                              events_dir=events_dir,
#                               masker_dir=masker_dir,
                              ses_id=ses)
            for ses in ["V03", "V10"]]

sessions = v03+v10

from load_confounds import CompCor, Minimal, Scrubbing

com_, min_, scr_ = CompCor(), Minimal(), Scrubbing()

comparison = dict(tuple((str(loader.__class__).split('.')[-1][:-2],
                         [loader.load(apath) for apath in
                          [s.fmri_path for s in sessions]])
                        for loader in tqdm_([min_, com_, scr_ ])))
# atlases_dir, fmriprep_dir, events_dir, masker_dir, labels_path, save_dir

  0%|          | 0/3 [00:00<?, ?it/s]

/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: Use

/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: Use

134

In [133]:
def get_json(src):
    with open(src, mode='r') as jfile:
        jobj = json.load(jfile)
    jfile.close()
    return jobj

metaconfs = [pd.read_json(apath)
             for apath in valid_meta]

In [149]:
display(metaconfs[0], allconf[0].filter(regex='displacement|outlier|rmsd|std_dvars|dvars|cosine'))

,a_comp_cor_00,a_comp_cor_01,a_comp_cor_02,a_comp_cor_03,a_comp_cor_04,a_comp_cor_05,a_comp_cor_06,a_comp_cor_07,a_comp_cor_08,a_comp_cor_09,...,dropped_96,dropped_97,dropped_98,dropped_99,global_signal,t_comp_cor_00,t_comp_cor_01,t_comp_cor_02,tcompcor,white_matter
CumulativeVarianceExplained,0.238389,0.436563,0.526639,0.144543,0.210335,0.258575,0.284986,0.306645,0.324943,0.341678,...,0.972045,0.972686,0.97332,0.973943,NaN,0.370144,0.469682,0.544861,NaN,NaN
Mask,CSF,CSF,CSF,WM,WM,WM,WM,WM,WM,WM,...,CSF,CSF,CSF,CSF,NaN,NaN,NaN,NaN,NaN,NaN
Method,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,...,aCompCor,aCompCor,aCompCor,aCompCor,Mean,tCompCor,tCompCor,tCompCor,Mean,Mean
Retained,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,NaN,True,True,True,NaN,NaN
SingularValue,131.849446,120.214932,81.047453,257.624866,173.809054,148.830901,110.125062,99.724824,91.661615,87.661082,...,6.856265,6.832528,6.801996,6.740745,NaN,321.905635,166.931642,145.074719,NaN,NaN
VarianceExplained,0.238389,0.198174,0.090076,0.144543,0.065791,0.04824,0.026412,0.021659,0.018298,0.016735,...,0.000645,0.00064,0.000634,0.000623,NaN,0.370144,0.099538,0.075179,NaN,NaN


,std_dvars,dvars,framewise_displacement,rmsd,cosine00,cosine01,cosine02,cosine03,cosine04,cosine05,...,motion_outlier07,motion_outlier08,motion_outlier09,motion_outlier10,motion_outlier11,motion_outlier12,motion_outlier13,motion_outlier14,motion_outlier15,motion_outlier16
0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.351688,28.792343,0.064201,0.047316,0.080451,0.080448,0.080442,0.080435,0.080426,0.080414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.275222,27.163544,0.156896,0.091658,0.080442,0.080414,0.080368,0.080302,0.080218,0.080115,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.071430,22.822565,0.034885,0.018701,0.080426,0.080348,0.080218,0.080036,0.079803,0.079518,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.088482,23.185787,0.045809,0.031850,0.080401,0.080248,0.079994,0.079638,0.079182,0.078625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,1.116210,23.776428,0.091619,0.054641,-0.080368,0.080115,-0.079695,0.079108,-0.078356,0.077440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306,1.330451,28.339970,0.231160,0.131482,-0.080401,0.080248,-0.079994,0.079638,-0.079182,0.078625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307,1.363225,29.038094,0.144173,0.067794,-0.080426,0.080348,-0.080218,0.080036,-0.079803,0.079518,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,1.164965,24.814959,0.071352,0.034153,-0.080442,0.080414,-0.080368,0.080302,-0.080218,0.080115,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [161]:
# from load_confounds import CompCor, Minimal, Scrubbing

min_ = Minimal(global_signal="full", wm_csf="full")
"""
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235:
UserWarning: Numerical issues were encountered when centering the data and might not be solved.
Dataset may contain too large values. You may need to prescale your features.

"""
[min_.load(apath) for apath in
 sorted(s.fmri_path for s in sessions)]
# min_.load(sorted(s.fmri_path for s in sessions)[0])
# sorted(min_.__dict__.keys())
# reduced_ = 


# allconf[0].filter(regex='outlier|csf|wm|global_signal|framewise|displacement')


/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: Use

/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:235: Use

['columns_',
 'confounds_',
 'demean',
 'global_signal',
 'json_',
 'missing_confounds_',
 'missing_keys_',
 'motion',
 'n_motion',
 'strategy',
 'wm_csf']

In [182]:
pd.DataFrame(min_.load(sessions[5].fmri_path),
             columns=min_.columns_)
# min_.columns_.shape

,cosine00,cosine01,cosine02,cosine03,cosine04,cosine05,cosine06,cosine07,cosine08,cosine09,...,csf_derivative1,csf_power2,csf_derivative1_power2,white_matter_derivative1,white_matter_power2,white_matter_derivative1_power2,global_signal,global_signal_derivative1,global_signal_power2,global_signal_derivative1_power2
0,-2.865092e-18,-3.225821e-13,-1.432546e-18,3.225803e-13,-1.432546e-18,6.774189e-12,7.162729e-19,3.225787e-13,-7.162729e-19,-3.225787e-13,...,-22.465774,68844.029900,442.912842,2.169908,-8070.574152,2.356069,1.405329,-3.901961,2546.099456,10.749246
1,8.045076e-02,8.044764e-02,8.044244e-02,8.043517e-02,8.042581e-02,8.041438e-02,8.040087e-02,8.038528e-02,8.036761e-02,8.034787e-02,...,-22.465774,10188.084528,442.912842,2.169908,-3943.730505,2.356069,-2.494152,-3.901961,-4585.816354,10.749246
2,8.044244e-02,8.041438e-02,8.036761e-02,8.030216e-02,8.021802e-02,8.011523e-02,7.999381e-02,7.985378e-02,7.969519e-02,7.951805e-02,...,-3.407083,1060.030956,-55.828791,-1.837973,-7308.081195,0.719345,-4.346488,-1.854816,-7962.971912,-1.025559
3,8.042581e-02,8.034787e-02,8.021802e-02,8.003635e-02,7.980298e-02,7.951805e-02,7.918176e-02,7.879431e-02,7.835596e-02,7.786699e-02,...,-1.035780,-1973.727003,-67.065760,-2.082893,-11122.409780,1.660921,-3.728150,0.615857,-6836.390070,-4.074364
4,8.040087e-02,8.024814e-02,7.999381e-02,7.963820e-02,7.918176e-02,7.862506e-02,7.796881e-02,7.721385e-02,7.636113e-02,7.541172e-02,...,-9.352255,-26220.404426,21.786643,-2.196521,-15139.639399,2.138495,-5.930562,-2.204893,-10845.594837,0.393914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,-8.036761e-02,8.011523e-02,-7.969519e-02,7.910835e-02,-7.835596e-02,7.743958e-02,-7.636113e-02,7.512287e-02,-7.372740e-02,7.217762e-02,...,-7.849489,39304.131877,-4.508158,1.082437,11480.858288,-1.263890,5.007730,-0.152822,9161.699771,-4.434103
306,-8.040087e-02,8.024814e-02,-7.999381e-02,7.963820e-02,-7.918176e-02,7.862506e-02,-7.796881e-02,7.721385e-02,-7.636113e-02,7.541172e-02,...,6.940855,56969.467084,-22.323162,-0.281765,11021.214158,-2.460449,6.437976,1.427765,11795.460811,-2.411102
307,-8.042581e-02,8.034787e-02,-8.021802e-02,8.003635e-02,-7.980298e-02,7.951805e-02,-7.918176e-02,7.879431e-02,-7.835596e-02,7.786699e-02,...,-12.379454,24463.197818,88.468514,-0.441312,10260.716494,-2.357281,8.962120,2.521663,16453.588333,1.914597
308,-8.044244e-02,8.041438e-02,-8.036761e-02,8.030216e-02,-8.021802e-02,8.011523e-02,-7.999381e-02,7.985378e-02,-7.969519e-02,7.951805e-02,...,-0.293911,23322.422560,-68.271712,0.034381,10397.667341,-2.514492,9.107850,0.143250,16722.912702,-4.435468


In [188]:
tst = sessions[0].fmri_path

display(tst, pd.DataFrame(min_.load(tst), columns=min_.columns_))

# == min_.confounds_, min_.columns_.shape
# min_._parse

# min_.columns_.tolist()

'/data/simexp/cimaq_preproc/fmriprep/sub-3025432/ses-V03/func/sub-3025432_ses-V03_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

,cosine00,cosine01,cosine02,cosine03,cosine04,cosine05,cosine06,cosine07,cosine08,cosine09,...,csf_derivative1,csf_power2,csf_derivative1_power2,white_matter_derivative1,white_matter_power2,white_matter_derivative1_power2,global_signal,global_signal_derivative1,global_signal_power2,global_signal_derivative1_power2
0,-2.865092e-18,-3.225821e-13,-1.432546e-18,3.225803e-13,-1.432546e-18,6.774189e-12,7.162729e-19,3.225787e-13,-7.162729e-19,-3.225787e-13,...,-38.976616,70065.638226,1474.139063,-3.798367,-2533.181655,10.053910,5.125620,-8.410377,8927.465491,62.269447
1,8.045076e-02,8.044764e-02,8.044244e-02,8.043517e-02,8.042581e-02,8.041438e-02,8.040087e-02,8.038528e-02,8.036761e-02,8.034787e-02,...,-38.976616,-23241.266796,1474.139063,-3.798367,-8941.428977,10.053910,-3.317775,-8.410377,-5782.774852,62.269447
2,8.044244e-02,8.041438e-02,8.036761e-02,8.030216e-02,8.021802e-02,8.011523e-02,7.999381e-02,7.985378e-02,7.969519e-02,7.951805e-02,...,-2.261993,-28903.182889,-51.274587,1.275298,-6785.505486,-2.713203,-5.179769,-1.828977,-9007.587180,-5.554440
3,8.042581e-02,8.034787e-02,8.021802e-02,8.003635e-02,7.980298e-02,7.951805e-02,7.918176e-02,7.879431e-02,7.835596e-02,7.786699e-02,...,-6.611238,-44692.873934,-11.337769,1.106738,-4911.136228,-3.115851,0.123358,5.336144,195.217861,19.101692
4,8.040087e-02,8.024814e-02,7.999381e-02,7.963820e-02,7.918176e-02,7.862506e-02,7.796881e-02,7.721385e-02,7.636113e-02,7.541172e-02,...,3.568908,-36736.290975,-45.457254,0.590199,-3907.917927,-3.995857,-1.406895,-1.497236,-2466.088209,-6.679785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,-8.036761e-02,8.011523e-02,-7.969519e-02,7.910835e-02,-7.835596e-02,7.743958e-02,-7.636113e-02,7.512287e-02,-7.372740e-02,7.217762e-02,...,9.487693,34792.728366,29.991628,0.818252,7884.369540,-3.673122,2.262183,-1.685999,3922.755075,-6.066443
306,-8.040087e-02,8.024814e-02,-7.999381e-02,7.963820e-02,-7.918176e-02,7.862506e-02,-7.796881e-02,7.721385e-02,-7.636113e-02,7.541172e-02,...,-2.263643,29008.508752,-51.266608,-0.401757,7205.396050,-4.189448,-0.393943,-2.623109,-704.959432,-1.966452
307,-8.042581e-02,8.034787e-02,-8.021802e-02,8.003635e-02,-7.980298e-02,7.951805e-02,-7.918176e-02,7.879431e-02,-7.835596e-02,7.786699e-02,...,-3.210154,20979.799663,-45.792901,-2.313751,3274.141535,0.989737,2.322226,2.749186,4027.528172,-1.643888
308,-8.044244e-02,8.041438e-02,-8.036761e-02,8.030216e-02,-8.021802e-02,8.011523e-02,-7.999381e-02,7.985378e-02,-7.969519e-02,7.951805e-02,...,2.279525,26047.410526,-52.599394,1.137971,5214.843999,-3.045531,4.173385,1.884177,7261.330694,-5.594670


In [48]:
, #[ses.fmri_path for ses in sessions]

# np.array(conf_pathlist)

[np.argsort(np.array(to_validate), kind='')]

Help on function argsort in module numpy:

argsort(a, axis=-1, kind=None, order=None)
    Returns the indices that would sort an array.
    
    Perform an indirect sort along the given axis using the algorithm specified
    by the `kind` keyword. It returns an array of indices of the same shape as
    `a` that index data along the given axis in sorted order.
    
    Parameters
    ----------
    a : array_like
        Array to sort.
    axis : int or None, optional
        Axis along which to sort.  The default is -1 (the last axis). If None,
        the flattened array is used.
    kind : {'quicksort', 'mergesort', 'heapsort', 'stable'}, optional
        Sorting algorithm. The default is 'quicksort'. Note that both 'stable'
        and 'mergesort' use timsort under the covers and, in general, the
        actual implementation will vary with data type. The 'mergesort' option
        is retained for backwards compatibility.
    
        .. versionchanged:: 1.15.0.
           The 'stab

In [39]:
pd.read_json(valid_meta[33])

,a_comp_cor_00,a_comp_cor_01,a_comp_cor_02,a_comp_cor_03,a_comp_cor_04,a_comp_cor_05,a_comp_cor_06,a_comp_cor_07,a_comp_cor_08,a_comp_cor_09,...,dropped_96,dropped_97,dropped_98,dropped_99,global_signal,t_comp_cor_00,t_comp_cor_01,t_comp_cor_02,tcompcor,white_matter
CumulativeVarianceExplained,0.336394,0.472346,0.551729,0.101283,0.187765,0.239271,0.270285,0.298712,0.319545,0.335979,...,0.970773,0.971595,0.972407,0.973207,NaN,0.262864,0.437425,0.572312,NaN,NaN
Mask,CSF,CSF,CSF,WM,WM,WM,WM,WM,WM,WM,...,CSF,CSF,CSF,CSF,NaN,NaN,NaN,NaN,NaN,NaN
Method,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,aCompCor,...,aCompCor,aCompCor,aCompCor,aCompCor,Mean,tCompCor,tCompCor,tCompCor,Mean,Mean
Retained,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,NaN,True,True,True,NaN,NaN
SingularValue,139.791909,88.86926,67.908313,160.39264,148.209962,114.378174,88.755051,84.972571,72.744211,64.607783,...,7.011443,6.909028,6.86772,6.817931,NaN,287.553302,234.329822,205.986215,NaN,NaN
VarianceExplained,0.336394,0.135952,0.079383,0.101283,0.086482,0.051506,0.031014,0.028427,0.020834,0.016434,...,0.000846,0.000822,0.000812,0.0008,NaN,0.262864,0.174562,0.134887,NaN,NaN


In [ ]:
Path('/data/simexp/fnadeau/cimaq_confounds_list.txt').write_text('\n'.join(sorted(map(str, valid_conflist))),
                                                                 encoding='UTF-8-SIG')

In [ ]:
loaded_valid[0].filter(regex='outlier').replace(0, np.nan).dropna(axis=0, how='all')

In [ ]:


# dst = '/data/simexp/fnadeau/cimaq_computed_data/'

# valid = [ses for ses in sessions if get_sub_ses_key(ses.fmri_path)
#          not in list(map(get_sub_ses_key, os.listdir(dst)))]

# selected = sample(valid, 10)


# dst = '/data/simexp/fnadeau/cimaq_computed_data/'

# def Ses2Disk(sessions, dst):
# #     process = Process(target=Ses2Disk,
# #                       name=None,
# #                       args=(sessions, dst))
# #     process.start()    
#     import pickle
#     from io import BytesIO

#     os.makedirs(dst, exist_ok=True)
#     SesGen = yield from (fetch_fmriprep_session(session=ses)
#                          for ses in selected)
    
#     for ses in tqdm_(sessions):
#         name = '_'.join([ses.sub_id, ses.ses_id, ses.task,
#                          'computed-data.pickle'])
#         with open(os.path.join(dst, name), mode='wb') as cornichon:
#             pickle.dump(next(SesGen), cornichon, protocol=5)
#             cornichon.close()
# #     process.stop()
        
    



# Ses2Disk(sessions=selected, dst=dst)

# for ses in tqdm_(sess5):
#     name = '_'.join([ses.sub_id, ses.ses_id, ses.task,
#                      'computed-data.pickle'])
#     with open(os.path.join(dst, name), mode='wb') as cornichon:
#         pickle.dump(ses, cornichon, protocol=5)
    
# # display(there, new, [itm for itm in new if itm in there])
# sess5 = [fetch_fmriprep_session(session=ses)
#          for ses in tqdm_(sample(v03+v10, 5))]

In [ ]:
len(os.listdir('/data/simexp/fnadeau/cimaq_computed_data/'))

In [ ]:
import pickle
from io import BytesIO

dst = '/data/simexp/fnadeau/cimaq_computed_data/'
os.makedirs(dst, exist_ok=True)

for ses in tqdm_(sess5):
    name = '_'.join([ses.sub_id, ses.ses_id, ses.task,
                     'computed-data.pickle'])
    with open(os.path.join(dst, name), mode='wb') as cornichon:
        pickle.dump(ses, cornichon, protocol=5)
    cornichon.close()

In [ ]:
# to_validate = sorted(f'{s.sub_id}_{s.ses_id}'
#                                for s in sessions)


In [ ]:

def processtest(ses_ids: list):
    process = Process(target=processtest,
                      name=None,
                      args=(['V03', 'V10']))
    process.start()
    v03, v10 = [get_fmri_sessions(topdir=fmriprep_dir,
                                  events_dir=events_dir,
                                  masker_dir=None,
                                  ses_id=ses)
                for ses in tqdm_(['V03', 'V10'])]

    sessions = v03+v10
    return sessions

# print(len(v03), len(v10))
sessions=processtest(ses_ids=['V03', 'V10'])




In [ ]:
from cimaq_decoding_utils import get_sub_ses_key

get_sub_ses_key(sessions[55].fmri_path)

In [240]:
import html5lib
import inspect
from inspect import getmembers
from html.parser import HTMLParser

from bs4 import BeautifulSoup as bs
from io import BytesIO, StringIO
from markdownify import markdownify as markdownify_
from IPython.display import HTML

from html.parser import HTMLParser
# HTML(s.set_table_attributes("border=1").render())



to_validate = sorted(map(list, sorted(map(get_sub_ses_key, [sess.fmri_path for sess
                             in sessions]))))
# from cimaq_decoding_utils import get_sub_ses_key

html_dir = Path('/data/simexp/fnadeau/cimaq_html/')
html_txt_dir = Path('/data/simexp/fnadeau/cimaq_html2txt/')
html_list = sorted(html_dir.rglob('*.html'))


conf_dir = Path('/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/')
conf_pathlist = list(map(str, list(Path(conf_dir).iterdir())))
conf_ids = list(map(list, list(map(get_sub_ses_key, conf_pathlist))))

conf_inventory = pd.DataFrame([sorted(alist) for alist in
                               [conf_pathlist, conf_ids]])

allconf = [pd.read_csv(apath, sep='\t') for apath in conf_inventory.loc[0]]

# Counter([t.shape for t in allconf])
# 
# to_validate = sorted([s.sub_id, s.ses_id]
#                      for s in sessions)





# valid_confs = [p[1] for p in enumerate(list(Path(conf_dir).iterdir()))
#                       if conf_ids[p[0]] in to_validate]
# display(len(to_validate), len(valid_confs))

metaconf_dir = '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds_meta/'
meta_pathlist = list(map(str, list(Path(metaconf_dir).iterdir())))
meta_indexes = list(map(list, list(map(get_sub_ses_key, meta_pathlist))))
valid_meta = [p[1] for p in enumerate(meta_pathlist)
              if meta_indexes[p[0]] in to_validate]

len(valid_meta)
#  if itm in to_validate]
# conf_idxs = ['_'.join(idx) for idx in sorted(map(get_sub_ses_key, sorted(conf_dir.iterdir())))]

# to_validate
# valid_conf = set(list(map(get_sub_ses_key, 
#                         list(map(str, )))))

# len(valid_conf)
# valid_conflist = [apath for apath in
#                   tqdm_(sorted(conf_dir.iterdir()))
#                   if '_'.join(get_sub_ses_key(apath)) in to_validate]

# loaded_valid = [pd.read_csv(apath, sep='\t') for apath in valid_conflist]

# hprs = HTMLParser()
# hprs.feed(data=html_list[0].read_text())
# hprs.handle_data(data=html_list[0].read_text())
# # sorted(dir(hprs))

# from IPython.core.display import display, HTML
# display(HTML(hprs.rawdata))
# # HTMLParser().__dir__()

# # print(html_list[0].read_text().replace("./sub-3002498/", "./"))
# #.parse(html_list[0].read_text())
# # os.makedirs(html_txt_dir, exist_ok=True)


# # HTML(filename=html_list[0])#BytesIO(html_list[0].read_bytes().replace(b"./sub-3002498/", b"./")).read())

134

In [345]:
# sorted(min_.__dict__.keys())
# # min_.n_motion,
# min_.columns_, min_.confounds_.shape
# # min_.missing_confounds_, min_.missing_keys_, min_.motion

# 'non_steady_state_outlier_XX'

[c[['framewise_displacement','rmsd', 'dvars', 'std_dvars']]
 for c in allconf]

# pd.concat([conf[['framewise_displacement']] for conf in allconf], axis=1).describe()


[     framewise_displacement      rmsd      dvars  std_dvars
 0                       NaN       NaN        NaN        NaN
 1                  0.064201  0.047316  28.792343   1.351688
 2                  0.156896  0.091658  27.163544   1.275222
 3                  0.034885  0.018701  22.822565   1.071430
 4                  0.045809  0.031850  23.185787   1.088482
 ..                      ...       ...        ...        ...
 305                0.091619  0.054641  23.776428   1.116210
 306                0.231160  0.131482  28.339970   1.330451
 307                0.144173  0.067794  29.038094   1.363225
 308                0.071352  0.034153  24.814959   1.164965
 309                0.062135  0.034410  25.138691   1.180163
 
 [310 rows x 4 columns],
      framewise_displacement      rmsd      dvars  std_dvars
 0                       NaN       NaN        NaN        NaN
 1                  0.061121  0.039340  30.439184   1.272876
 2                  0.060561  0.038064  26.696356   1.1163

In [400]:
import warnings

# help(warnings.filterwarnings)


array(['/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3002498_ses-V10_task-memory_desc-confounds_timeseries.tsv',
       '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3025432_ses-V03_task-memory_desc-confounds_timeseries.tsv',
       '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3025432_ses-V10_task-memory_desc-confounds_timeseries.tsv',
       '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3123186_ses-V03_task-memory_desc-confounds_timeseries.tsv',
       '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3123186_ses-V10_task-memory_desc-confounds_timeseries.tsv',
       '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3149469_ses-V03_task-memory_desc-confounds_timeseries.tsv',
       '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3163875_ses-V03_task-memory_desc-confounds_timeseries.tsv',
       '/data/simexp/fnadeau/cimaq_fmriprep_outputs/cimaq_confounds/sub-3186

In [317]:
def make_confounds(fmri_path,
                   strategy: str = 'Minimal',
                   motion: str = 'basic',
                   wm_csf: str = 'basic',
                   global_signal: str = 'basic',
                   demean: bool = True,
                   **kwargs):
    
    from inspect import getmembers
    import load_confounds
    import warnings
    from builtins import UserWarning
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    if kwargs is None:
        kwargs = {}
    loader = dict(getmembers(load_confounds))[f'{strategy}']
    loader = loader(motion=motion, wm_csf=wm_csf,
                    global_signal=global_signal,
                    demean=demean,
                     **kwargs)

    with warnings.catch_warnings(record=False) as w:
#     with warnings.filterwarnings("ignore", category=UserWarning) as w:
        try:
            data = pd.DataFrame(loader.load(fmri_path))
        except UserWarning:
            data = pd.DataFrame(scaler.fit_transform(loader.load(fmri_path)))
        if hasattr(loader, 'columns_'):
            data.set_axis(loader.columns_, axis=1, inplace=True)
    setattr(loader, 'confounds_table', data)
    return loader
        
confounds = [make_confounds(ses.fmri_path,
                            strategy='Scrubbing',
                            scrub='basic',
                            motion='basic',
                            wm_csf='basic',
                            global_signal='basic')
             for ses in sessions]
# confounds

In [367]:
help(getattr)

Help on built-in function getattr in module builtins:

getattr(...)
    getattr(object, name[, default]) -> value
    
    Get a named attribute from an object; getattr(x, 'y') is equivalent to x.y.
    When a default argument is given, it is returned when the attribute doesn't
    exist; without it, an exception is raised in that case.



In [452]:
mini = Minimal(global_signal='full',wm_csf='full', motion='full')
confs = mini.load(fmri_paths)

scrub = Scrubbing(global_signal='full',
                  wm_csf='full', motion='full',
                  std_dvars_thresh=1.5,
                  fd_thresh=0.5, scrub='full')
s_confounds = scrub.load(fmri_paths)
# scrubbed = [mini._load_scrub(confounds_raw=c) for c in allconf]
# scrubbed = mini._load_scrub(confounds_raw=allconf)

# mini_motions_outs = [c.filter(regex='framewise_displacement|rmsd|dvars|non_steady_state_outlier*')

#                for c in allconf]
# scrub_motions_outs = [c.filter(regex='framewise_displacement|rmsd|dvars|non_steady_state_outlier*')

#                for c in allconf]
# display(*motions_outs)


In [442]:
from load_confounds import Scrubbing, CompCor


out_indexes = [mo['framewise_displacement'].values<0.5
               for mo in motions_outs]
# tuple(enumerate(list(map(len, out_indexes))))

# motions_outs[57]
scrubbed = [scrub._load_scrub(confounds_raw=c)
            for c in motions_outs]
# out_indexes
[s.shape for s in scrubbed]

[(310, 20),
 (310, 21),
 (310, 31),
 (310, 6),
 (310, 27),
 (310, 16),
 (310, 298),
 (288, 143),
 (310, 185),
 (310, 30),
 (310, 94),
 (310, 108),
 (310, 15),
 (310, 89),
 (310, 25),
 (310, 19),
 (310, 28),
 (310, 14),
 (310, 33),
 (310, 127),
 (310, 151),
 (288, 131),
 (310, 27),
 (310, 33),
 (310, 98),
 (310, 257),
 (310, 187),
 (310, 109),
 (310, 164),
 (254, 27),
 (310, 11),
 (310, 198),
 (310, 59),
 (310, 198),
 (310, 148),
 (310, 20),
 (288, 152),
 (310, 275),
 (310, 46),
 (310, 305),
 (288, 147),
 (310, 219),
 (310, 237),
 (310, 287),
 (310, 84),
 (310, 22),
 (310, 231),
 (310, 53),
 (288, 288),
 (310, 44),
 (310, 105),
 (310, 99),
 (310, 169),
 (310, 89),
 (310, 105),
 (310, 53),
 (310, 48),
 (60, 13),
 (288, 235),
 (310, 37),
 (310, 247),
 (288, 288),
 (288, 288),
 (310, 98),
 (310, 5),
 (288, 58),
 (310, 9),
 (310, 27),
 (310, 59),
 (310, 49),
 (310, 26),
 (310, 97),
 (310, 54),
 (310, 72),
 (310, 214),
 (310, 54),
 (310, 22),
 (310, 152),
 (310, 30),
 (310, 100),
 (310, 140)

In [467]:
#!/usr/bin/python3

"""
Compute the TSNR of a bunch of nifti files and generate the equivalent nifti
SNR 3Ds.

Depends on nibabel, matplotlib and scipy

"""

import os
import sys
from glob import glob
import getopt

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import nanmean
from scipy.io import savemat

from nibabel import load, save
import nibabel.nifti1 as nifti


def tsnr(data, affine, file_name
         ):

    mean_d = np.mean(data, axis=-1)
    std_d = np.std(data, axis=-1)
    tsnr = mean_d/std_d
    tsnr[np.where(np.isinf(tsnr))] = np.nan
    mean_tsnr = nanmean(np.ravel(tsnr))
    tsnr_image = nifti.Nifti1Image(tsnr, affine)
    save(tsnr_image, file_name)
    savemat(file_name.split('.')[0], {'tsnr': tsnr})
    return mean_tsnr


    usage = "Usage: fmri_tsnr [options] nifti_dir"

    help_me = ' '.join(["This program computes the temporal SNR Input",
                        "is a directory of nifti files"])
    opts =  ["help", "cat", "verbose", "plot"]

# if __name__ == "__main__":
#     # Get inputs and options: Last argument is your input:
#     path = sys.argv[-1]

#     try:
#         opts, args = getopt.getopt(sys.argv[1:], "hcvp", opts)
#     except (getopt.GetoptError):
#         # print help information and exit:
#         raise getopt.GetoptError
#         print(usage)
#         sys.exit(2)
#     # Set the defaults:
#     cat, verbose, p = False, False, False

#     # Then get the input opts:
#     for o, a in opts:
#         if o in ["-v", "--verbose"]:
#             verbose = True
#         elif o in ["-h", "--help"]:
#             print(usage+'\n'+help_me)
#         elif o in ["-c", "--cat"]:
#             cat = True
#         elif o in ["-p", "--plot"]:
#             p = True
#         else:
#             print(usage)
#             assert False, "unhandled option"

#     # Make sure you have a place to save the results:
#     tsnr_path = '%s/TSNR/' % path
#     os.makedirs(tsnr_path, exist_ok=True)

    allepi = glob(f'{path}*.nii*').sort()
    # If you are concatenating initialize the data with the first one:
    if cat:
        epi1 = load(allepi.pop(0))
        data = epi1.get_data()
        affine = epi1.get_affine()

    # Otherwise, you might want to plot the average tsnr on a scan-by-scan
    # basis:
    else:
        snr, label = [], []
    for epi in allepi:
        if verbose:
            print(f'Groking {epi}')

        # If you are concatenating, just concatenate:
        if cat:
            data = np.concatenate([data, load(epi).get_data()], axis=-1)

        # Otherwise, do the analysis on a file-by-file basis:
        else:
            fname = os.path.split(epi)[-1].split('.')[0]
            nibber = load(epi)
            affine = nibber.get_affine()
            data = nibber.get_data()
            snr.append(tsnr(data, affine, '%s%s_tsnr.nii.gz' %
                       (tsnr_path, fname)))
            label.append(fname)
    # Then, if you were cat'ing, do the analysis, when you exit the cat-loop:
    if cat:
        snr = tsnr(data, affine, '%stsnr_mean.nii.gz' % tsnr_path)

    if p == True:
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.bar(np.arange(len(snr))+1, snr)
        ax.set_xticks(np.arange(len(snr))+1)
        ax.set_xticklabels(label)
        labels = ax.get_xticklabels()
        ax.set_ylabel('SNR')
        ax.set_xlabel('File')
        fig.set_size_inches(len(label)*1.2, 8)
        plt.setp(labels, rotation=45, fontsize=10)
        fig.savefig('%smean_tsnr_.png' % tsnr_path)


IndentationError: unexpected indent (118913959.py, line 77)

In [443]:
from load_confounds import Minimal

fmri_paths = sorted(s.fmri_path for s in sessions)



# '_load_compcor',, '_load_confound',


functions = [
             '_load_global',
             '_load_high_pass',
             '_load_ica_aroma',
             '_load_motion',
             '_load_scrub',
             '_load_single',
             '_load_wm_csf']
# sorted(dir(mini))
# help(mini._load_wm_csf)

# [getattr(mini, f)(mini, confounds_raw=allconf) for f in functions]

# [getattr(mini, f)(confounds_raw=allconf,
#                   confound=allconf) for f in functions]
# mini._load_compcor(confounds_raw=allconf)
# 
help(scrub._load_scrub)

Help on method _load_scrub in module load_confounds.parser:

_load_scrub(confounds_raw) method of load_confounds.strategies.Scrubbing instance
    Perform basic scrub - Remove volumes if framewise displacement exceeds threshold.



In [470]:
mincols = sorted(set(flatten([ind.tolist() for ind in mini.columns_])))
scrubcols = sorted(set(flatten([ind.tolist() for ind in scrub.columns_])))

import cmake
print(cmake.__file__)

/home/fnadeau/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/cmake/__init__.py


In [469]:
def tsnr(img,
         file_name: str = None
         ):

    img = nimage.load_img(img)
    affine = img.affine
    mean_d = np.mean(data, axis=-1)
    std_d = np.std(data, axis=-1)
    tsnr = mean_d/std_d
    tsnr[np.where(np.isinf(tsnr))] = np.nan
    return nanmean(np.ravel(tsnr))

tsnr(fmri_paths[3])
#     tsnr_image = nifti.Nifti1Image(tsnr, affine)
#     save(tsnr_image, file_name)
#     savemat(file_name.split('.')[0], {'tsnr': tsnr})
#     return mean_tsnr

NameError: name 'data' is not defined

In [466]:
# len(mincols), len(scrubcols)

# [sorted(ldc.__dict__.keys()) for ldc in [mini,scrub]]
([sc.shape for sc in scrub.confounds_], np.array(scrub.columns_).shape)

([(310, 67),
  (310, 68),
  (310, 53),
  (310, 74),
  (310, 63),
  (310, 345),
  (310, 232),
  (310, 77),
  (310, 155),
  (310, 62),
  (310, 136),
  (310, 72),
  (310, 66),
  (310, 61),
  (310, 80),
  (310, 174),
  (288, 177),
  (310, 74),
  (310, 79),
  (310, 145),
  (310, 304),
  (310, 234),
  (310, 156),
  (254, 71),
  (310, 245),
  (310, 106),
  (310, 195),
  (310, 67),
  (288, 198),
  (310, 322),
  (310, 93),
  (288, 193),
  (310, 266),
  (310, 284),
  (310, 334),
  (310, 131),
  (310, 69),
  (310, 278),
  (310, 100),
  (288, 334),
  (310, 91),
  (310, 152),
  (310, 100),
  (310, 95),
  (288, 281),
  (310, 84),
  (310, 294),
  (288, 334),
  (288, 334),
  (310, 145),
  (310, 52),
  (288, 104),
  (310, 56),
  (310, 74),
  (310, 106),
  (310, 96),
  (310, 73),
  (310, 101),
  (310, 119),
  (310, 261),
  (310, 69),
  (310, 199),
  (310, 77),
  (310, 147),
  (310, 121),
  (310, 59),
  (310, 101),
  (310, 156),
  (310, 313),
  (310, 87),
  (310, 284),
  (310, 92),
  (310, 141),
  (310, 

In [ ]:
# sorted(mini.__dict__.keys())
# ['framewise_displacement' in cols.tolist() for cols in mini.columns_]
from cimaq_decoding_utils import flatten

Counter(flatten(cols.tolist() for cols in mini.columns_)).most_common()

In [305]:
[itm.filter(regex='outlier|csf|global').sum() for itm in confounds]

[csf                                -1.705303e-11
 csf_derivative1                     3.552714e-14
 csf_power2                         -2.793968e-08
 csf_derivative1_power2              1.364242e-12
 motion_outlier_0                    0.000000e+00
                                         ...     
 motion_outlier_134                 -1.665335e-16
 global_signal                       3.126388e-11
 global_signal_derivative1          -5.329071e-15
 global_signal_power2               -6.286427e-09
 global_signal_derivative1_power2    0.000000e+00
 Length: 143, dtype: float64,
 csf                                 4.638423e-11
 csf_derivative1                     0.000000e+00
 csf_power2                          1.490116e-08
 csf_derivative1_power2              9.094947e-13
 motion_outlier_0                    2.220446e-16
                                         ...     
 motion_outlier_95                  -1.665335e-16
 global_signal                      -6.480150e-12
 global_signal_deriv

In [293]:
confounds[-1].filter(regex='outlier')

,motion_outlier_0,motion_outlier_1,motion_outlier_2,motion_outlier_3,motion_outlier_4,motion_outlier_5,motion_outlier_6,motion_outlier_7,motion_outlier_8,motion_outlier_9,...,motion_outlier_17,motion_outlier_18,motion_outlier_19,motion_outlier_20,motion_outlier_21,motion_outlier_22,motion_outlier_23,motion_outlier_24,motion_outlier_25,motion_outlier_26
0,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
1,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
2,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
3,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
4,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
306,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
307,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888
308,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,...,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888,-0.056888


In [189]:

# help(Minimal())
# [Minimal().load(str(apath)) for apath in valid_conflist]



# html2table = [pd.read_html(html_string) for html_string in html2txt]


# sorted(valid_conflist[0].columns.tolist())



# qc_confounds = pd.DataFrame(zip([s.fmri_path for s in sessions]))

# sorted(dict(getmembers(scr_)).keys())

# new_dst = [os.path.join(html_txt_dir,
#                         os.path.splitext(os.path.basename(apath))[0]+'.txt')
#                  for apath in sorted(map(str, html_list))
#            if get_sub_ses_key(apath) in to_validate]
            
            
# new_txt_bytes = [bs(StringIO(Path(apath).read_text())).get_text()#.encode('UTF-8-SIG')
#                  for apath in html_list]
# print(new_txt_bytes[0])

# print(f"{html_list[0]}_{bs(StringIO(html_list[0].read_text())).get_text()}")
#                  if get_sub_ses_key(apath) in to_validate]
# to_validate

In [331]:
# min_.__dict__

In [111]:
from markdownify import markdownify as md

dst = '/data/simexp/fnadeau/cimaq_html2md/'

os.makedirs(dst, exist_ok=True)
# html2md
[Path(os.path.join(dst, os.path.splitext(
    os.path.basename(apath))[0]+'.md'), mode='wb').write_bytes(md(html_list[0].read_text()).encode('UTF-8-SIG'))
 for apath in html_list]
# print(f"{html_list[0]}_{md(html_list[0].read_text())}")

[77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,
 77442,


In [ ]:
confpath = '/data/simexp/fnadeau/cimaq_confounds/'


valid_confounds = [apath for apath in sorted(Path(confpath).iterdir())
                   if get_sub_ses_key(apath) in to_validate]
len(valid_confounds), len(to_validate)

In [ ]:
# [Path(itm[0], mode='wb').write_bytes(itm[1])
#  for itm in tuple(zip(new_dst, new_txt_bytes))
#  if get_sub_ses_key(itm[0]) in to_validate]
dst = '/data/simexp/fnadeau/cimaq_computed_data/'
to_validate = [s.fmri_path for s in sessions if get_sub_ses_key(s.fmri_path)
               not in os.listdir(dst)]

validation = sorted(map(get_sub_ses_key, to_validate))
validation
# tst = sorted(map(get_sub_ses_key, ))
# tst
# sorted((map(get_sub_ses_key, tuple(Path(confpath).iterdir()))))

In [112]:
conf_meta

NameError: name 'conf_meta' is not defined

In [ ]:
# display(*html2txt)
# help(pd.read_html)
sorted(Path(os.getcwd()).rglob('difumo_atlases/1024/3mm/'))
# html_tables = [pd.read_html(apath) for apath in html_list]

In [ ]:
di1024 = get_difumo(data_dir=atlases_dir,
                    dimension=1024,
                    resolution_mm=3)

In [ ]:


def save_maskers(sessions, di1024, dst):
    import pickle
    process = Process(target=save_maskers,
                      args=(sessions, di1024, dst))
    process.start()    
    os.makedirs(dst, exist_ok=True)
    [pickle.dump(NiftiMapsMasker(maps_img=di1024.maps,
                                 mask_img=nimage.load_img(ses.mask_path),
                                 resampling_target='maps',
                                 **{'standardize': False,
                                    'standardize_confounds': False,
                                    'high_variance_confounds': False,
                                    'smoothing_fwhm': None,
                                    'detrend': False,
                                    'dtype': 'f',
                                    'low_pass': None,
                                    'high_pass': None,
                                    'allow_overlap': True}).fit(),
                open(os.path.join(dst, '_'.join([f'{ses.sub_id}',
                                                 f'{ses.ses_id}_task-{ses.task}',
                                                 '1024-maps-masker.pickle'])),
                     mode='wb'),
                protocol=5)
     for ses in tqdm_(sessions)]
      
save_maskers(sessions=sessions, di1024=di1024,
             dst='/data/simexp/fnadeau/cimaq_ses-V03_ses-V10_maps1024-maskers/')    

In [ ]:
print(sklearn.__file__)

In [ ]:
pd.DataFrame(processtest(ses_ids=['V03', 'V10'])).to_csv('
                                                         
                                                         
                                                         ',
                                                         sep='\t', encoding='UTF-8-SIG',index=True)

In [ ]:
def lookup(p: str) -> re.Pattern:
    import re
    return re.compile(f'(?<={p})[a-zA-Z0-9\W]*')

def get_parts(src):
    import re
    from os.path import basename
    from pathlib import Path
    
    from cimaq_decoding_utils import flatten

    bids_parts = ['sub-', 'ses-', 'task-',
                  'space-', 'desc-']
    return [itm.group() for itm in
            flatten([re.search(lookup(p),part)
                    for part in Path(basename(src)).parts
                    for p in bids_parts])]
#         return list(list(filter(None,list(re.search(lookup(p),part)
#                                           for part in
#                                           Path(basename(src)).parts)))
#                     for p in bids_parts)


def get_match(src: Union[str, PathLike, PosixPath],
              pattern: Union[str, re.Pattern]
              ) -> PosixPath:

    return sorted(Path(src).rglob(f'{pattern}'))

def by_index(X: Iterable,
             indexes: Iterable
             ) -> list:
    return [i[1] for i in enumerate(X) if i[0] in indexes]

class FMRIPrepPathMatcher:
    from glob import glob
    def __init__(self,
                 fmri_path,
                 anat_modality,
                 **kwargs):
        from os.path import basename, dirname, splitext

        self.fmri_path = fmri_path
        if kwargs is not None:
            [setattr(self, item[0], item[1])
             for item in tuple(kwargs.items())]
        self.anat_modality = anat_modality
        self.fmriprep_dir = Path(dirname(dirname(dirname(fmri_path))))
        self.fmri_dir = list(Path(fmri_path).parents)[3]
        self.ext = fmri_path.split('.', maxsplit=1)[1]
        bids_parts = ['sub_id', 'ses_id', 'task', 'space', 'desc']
        bids_values = basename(fmri_path).split('_')
        sub_id, ses_id, task, space, desc = bids_values

        anat_suffix = '_'.join([sub_id, f'*{space}_{desc}',
                                f'{anat_modality}.{ext}'])
        mask_suffix = '_'.join([sub_id, ses_id, task, space,
                                f'desc-brain_mask.{ext}'])
        events_suffix = '_'.join([sub_id, ses_id,
                                  task, 'events.tsv'])
        behav_suffix = '_'.join([sub_id, ses_id, task,
                                 'behavioural.tsv'])
########################################################################        
#         anat_suffix = '_'.join([f'{self.sub_id}', f'*{self.space}',
#                                 f'{self.desc}',
#                                 f'{anat_modality}.{self.ext}'])
#         mask_suffix = '_'.join([f'{self.sub_id}', f'*{self.ses_id}',
#                                 f'*{self.task}', f'*{self.space}',
#                                 f'desc-brain_mask.{self.ext}'])
#         self.events_suffix = '_'.join([f'{self.sub_id}',
#                                        f'{self.ses_id}',
#                                        f'{self.task}',
#                                        'events.tsv'])
#         self.behav_suffix = '_'.join([f'{self.sub_id}',
#                                        f'{self.ses_id}',
#                                        f'{self.task}',
#                                        'behavioural.tsv'])        
        self.anat_path = sorted(self.fmri_dir.rglob(anat_suffix))[0]
        
        self.mask_path = sorted(Path(self.fmri_dir).rglob(mask_suffix))[0]
        self.events_path = sorted(Path(self.events_dir).rglob(f'*{self.events_suffix}'))[0]
        self.behav_path = sorted(Path(self.events_dir).rglob(f'*{self.behav_suffix}'))[0]
        [setattr(self, itm[0], itm[1]) for itm
         in tuple(zip(bids_parts,bids_values))]

#      
FMRIPrepPathMatcher(sessions[0].fmri_path, 'T1w',
                    **{'events_dir': events_dir}).__dict__
# [x.group() for x in list(filter(None,flatten(PartMatcher(src))))]
# [getattr(itm[0], 'group')() for itm in PartMatcher(src)]

In [ ]:
# os.path.splitext(sessions[0].mask_path)
help(setattr)
# 'sub-3025432_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'
# 'sub-3025432_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz'

In [ ]:
# Counter([tuple(ses.keys()) for ses in sessions]).most_common()
os.path.basename(sessions[0].fmri_path).split('_')

In [ ]:
# class FMRIPrepPathMatcher():
#     def __init__(self, fmri_path):
#         fmri_path = fmri_path
#     self = __init__(fmri_path)
#     self.bids_parts = ['sub-', 'ses', 'task-',
#                        'space-', 'desc-']
#     self.get_parts = get_parts
from glob import glob


def get_sub_ses_key(fmri_path: Union[str, PathLike, PosixPath]
                    ) -> list:
    """
    Return a participant and session identifiers in a BIDS-compliant dataset.
    """

    return [p+next(iter(list(filter(None, (re.search(f'(?<={p})[a-zA-Z0-9]*',
                                                     part) for part in
                                           Path(fmri_path).parts))))).group()
            for p in ('sub-', 'ses-')]


def get_fmriprep_anat(fmri_path: Union[str, PathLike, PosixPath],
                      mdlt: str = 'T1w',
                      ext: str = 'nii.gz',
                      **kwargs):
    space = re.search(f'(?<=_space-)[a-zA-Z0-9]*',
                      basename(fmri_path)).group()
    anat_suffix = f'*_space-{space}_desc-preproc_{mdlt}.{ext}'
    return str(next(list(Path(fmri_path).parents)[2].rglob(anat_suffix)))


def get_events(fmri_path: Union[str, PathLike, PosixPath],
               events_dir: Union[str, PathLike, PosixPath]
               ) -> str:
    sub_id, ses_id = Path(fmri_path).parts[-4:-2]
    globbed = glob(os.path.join(events_dir,
                                *Path(fmri_path).parts[-4:-2],
                                '*events.tsv'))
    return [False if globbed == [] else globbed[0]][0]


def get_behav(fmri_path: Union[str, PathLike, PosixPath],
              events_dir: Union[str, PathLike, PosixPath]
              ) -> str:
    sub_id, ses_id = Path(fmri_path).parts[-4:-2]
    globbed = glob(os.path.join(events_dir,
                                *Path(fmri_path).parts[-4:-2],
                                '*behavioural.tsv'))
    return [False if globbed == [] else str(globbed[0])][0]


def get_fmriprep_mask(fmri_path: Union[str, PathLike, PosixPath],
                      mask_ext: str = 'nii.gz',
                      **kwargs):
    from os.path import basename, dirname
    def bids_patt(p): return f'(?<={p})[a-zA-Z0-9]*'
    prefixes = ['sub-', 'ses-', 'task-', 'space-']
    mask_sfx = '_'.join([pf+re.search(bids_patt(pf),
                                      basename(fmri_path)).group()
                         for pf in prefixes]+[f'desc-brain_mask.{mask_ext}'])
    return glob(os.path.join(dirname(fmri_path), mask_sfx))[0]

def get_maps_masker_path(fmri_path: Union[str, PathLike, PosixPath],
                    masker_dir: Union[str, PathLike, PosixPath],
                    dimension: int = 693,
                    resolution_mm: int = 3
                    ) -> NiftiMapsMasker:

    prefix = '_'.join(os.path.basename(session.fmri_path).split('_')[:-2])
    masker_path = sorted(Path(masker_dir).rglob(f'{prefix}*.pickle'))
    if masker_path == []:
        masker_path = None,
    else:
        masker_path = masker_path[0]
    return masker_path

In [ ]:
def get_fmri_sessions(topdir: Union[str, PathLike, PosixPath],
                      events_dir: Union[str, PathLike,
                                        PosixPath] = None,
                      masker_dir: Union[str, PathLike,
                                        PosixPath] = None,
                      task: str = 'memory',
                      space: str = 'MNI152NLin2009cAsym',
                      output_type: str = 'preproc',
                      extension: str = '.nii.gz',
                      modality: str = 'bold',
                      sub_id: str = '*',
                      ses_id: str = '*',
                      **kwargs
                      ) -> list:

    """
    Return a sorted list of the desired BOLD fMRI nifti file paths.

    Args:
        topdir: str, PathLike, or PosixPath (Default = None)
            Database top-level directory path.

        events_dir: str, PathLike, or PosixPath (Default = None)
            Directory where the events and/or behavioural
            files are stored. If None is provided, it is assumed
            to be identical as ``topdir``.

        masker_dir: str, PathLike, or PosixPath (Default = None)
            Directory where prefitted nilearn nifti maskers
            are located. Used to save fitting time.

        task: str (Default = 'memory')
            Name of the experimental task
            (i.e. 'rest' is also valid).

        space: str (Default = 'MNI152NLin2009cAsym')
            Name of the template used during resampling.
            Most likely corresponding to a valid TemplateFlow name.

        output_type: str (Default = 'preproc')
            Name of the desired FMRIPrep output type. Most likely
            corresponding to a valid FMRIPrep output type name.

        extension: str (Default = '.nii.gz')
            Nifti files extension. The leading '.' is required.

        modality: str (Default = 'bold')
            Scanning modality used during the experiment.

        sub_id: str (Default = '*')
            Participant identifier. By default, returns all
            participants. The leading 'sub-' must be omitted.
            If the identifier is numeric, it should be quoted.

        ses_id: str (Default = '*')
            Session identifier. By default, returns all sessions.
            If the identifier is numeric, it should be quoted.
            The leading 'ses-' must be omitted.

    Returns: list
        Sorted list of the desired nifti file paths.

    Notes:
        All parameters excepting ``topdir`` and ``events_dir``
        can be replaced by '*' (Default), which is
        equivalent to a UNIX ``find`` pattern.
    """
    
    from glob import glob
    from os.path import dirname
    from cimaq_decoding_utils import get_sub_ses_key, get_fmriprep_anat

    # Generate regex and glob patterns
    bold_pattern = '_'.join([f'sub-{sub_id}', f'ses-{ses_id}', f'task-{task}',
                             f'space-{space}', f'desc-{output_type}',
                             f'{modality}{extension}'])
    ev_pattern = f'sub-{sub_id}_ses-{ses_id}_task-{task}_events.tsv'

    # Load fMRI and events files paths into lists
    bold_paths = sorted(map(str, Path(topdir).rglob(f'*{bold_pattern}')))
    event_paths = sorted(map(str, (Path(events_dir).rglob(f'*{ev_pattern}'))))
    # Get only the intersection of these lists
    valid_bold_paths = sorted(boldpath for boldpath in bold_paths
                              if get_sub_ses_key(boldpath) in
                              [get_sub_ses_key(apath) for apath
                               in event_paths])
    valid_event_paths = sorted(evpath for evpath in event_paths
                               if get_sub_ses_key(evpath) in
                               [get_sub_ses_key(apath) for apath
                                in valid_bold_paths])
    # Load corresponding anatomical T1w, brain mask and behavioural file paths
    valid_anat_paths = [get_fmriprep_anat(v_boldpath) for
                        v_boldpath in valid_bold_paths]
    valid_mask_paths = [get_fmriprep_mask(v_boldpath) for
                        v_boldpath in valid_bold_paths]
    valid_behav_paths = [get_behav(v_boldpath, events_dir) for
                         v_boldpath in valid_bold_paths]

    # Zip them together
    zipped_paths = sorted(zip(valid_bold_paths, valid_anat_paths,
                              valid_mask_paths, valid_event_paths,
                              valid_behav_paths))

    # Create sklearn.utils.Bunch objects
    sessions = [Bunch(**dict(zip(['fmri_path', 'anat_path', 'mask_path',
                                  'events_path', 'behav_path',
                                  'sub_id', 'ses_id', 'task', 'space'],
                                 (item+Path(item[0]).parts[-4:-2] +
                                  ('task-'+task, 'space-'+space)))))
                for item in zipped_paths]
    
    return sessions

#     if masker_dir is not None:
#         [session.update({'masker_path':
#                          sorted(Path(masker_dir).rglob(f'{session.sub_id}_{session.ses_id}*.pickle'))[0]})
#          for session in sessions]
#     else:
#         [session.update({'masker_path': None}) for session in sessions]
    
#     return sessions

In [ ]:
def _pca_motion(confounds_motion, n_components):
    """
    Reduce the motion paramaters using PCA.
    """

    n_available = confounds_motion.shape[1]
    if n_components > n_available:
        raise ValueError(
            " ".join([f"User requested n_motion={n_components} motion",
                      "components, but found only {n_available}."])
        )
    confounds_motion = confounds_motion.dropna()
    confounds_motion_std = scale(
        confounds_motion, axis=0, with_mean=True, with_std=True
    )
    pca = PCA(n_components=n_components)
    motion_pca = pd.DataFrame(pca.fit_transform(confounds_motion_std))
    motion_pca.columns = ["motion_pca_" + str(col + 1)
                          for col in motion_pca.columns]
    return motion_pca

In [ ]:
get_fmri_sessions(topdir=fmriprep_dir,
                  events_dir=events_dir,
                  masker_dir=None,
                  ses_id="V10")[]

In [ ]:
sessions.__len__()

In [ ]:
# di1024 = get_difumo(data_dir=atlases_dir,
#                     dimension=1024,
#                     resolution_mm=3)

def MaskerGen(**kwargs):
    while True:
        yield NiftiMapsMasker(maps_img=di1024.maps,
                              mask_img=session.mask_img,
                              t_r=get_t_r(session.fmri_img),
                              resampling_target='mask',
                              **session.masker_defs).fit()

common_maps_masker = NiftiMapsMasker(maps_img=di1024.maps,
                                     resampling_target='maps',
                                     **{'standardize': False,
                                        'standardize_confounds': False,
                                        'high_variance_confounds': False,
                                        'smoothing_fwhm': None,
                                        'detrend': False,
                                        'dtype': 'f',
                                        'low_pass': None,
                                        'high_pass': None,
                                        'allow_overlap': True})

In [ ]:
di1024 = get_difumo(data_dir=atlases_dir,
                    dimension=1024,
                    resolution_mm=3)

In [ ]:

def MaskerGen(**kwargs):
    from cimaq_decoding_utils import get_t_r
    from get_difumo import get_difumo

    defs = {'t_r': yield next(get_t_r(img))
            'mask_img': None,
            'standardize': False,
            'standardize_confounds': False,
            'high_variance_confounds': False,
            'smoothing_fwhm': None,
            'detrend': False,
            'dtype': 'f',
            'low_pass': None,
            'high_pass': None,
            'allow_overlap': True}
    

    atlas = get_difumo(data_dir=atlases_dir,
                           dimension=1024,
                           resolution_mm=3)    
    while True:
        
        yield NiftiMapsMasker(maps_img=di1024.maps,
                              mask_img=session.mask_img,
                              t_r=get_t_r(session.fmri_img),
                              resampling_target='mask',
                              **session.masker_defs).fit()

In [ ]:




common_maps_masker = NiftiMapsMasker(maps_img=di1024.maps,
                                     resampling_target='maps',
                                     **{'standardize': False,
                                        'standardize_confounds': False,
                                        'high_variance_confounds': False,
                                        'smoothing_fwhm': None,
                                        'detrend': False,
                                        'dtype': 'f',
                                        'low_pass': None,
                                        'high_pass': None,
                                        'allow_overlap': True}).fit()


In [ ]:
from builtins import FutureWarning
import warnings
labels_path = '/data/simexp/fnadeau/cortex-difumo-693-labels.txt'
cortex_labels = Path(labels_path).read_text().splitlines()
warnings.filterwarnings(action='ignore', category=FutureWarning)

trial_type_cols=['trial_type', 'recognition_performance',
                 'ctl_miss_ws_cs']

test = get_all_contrasts(session=fetch_fmriprep_session(session=sessions[0]),
                         output_type='effect_size',
                         trial_type_cols=trial_type_cols,
                         standardize=True,
                         scale=False,
                         maximize=False,
                         feature_labels=cortex_labels,
                         extract_only=True)

In [ ]:
test

In [ ]:

# print(len(cortex_names))

from cimaq_decoding_utils import save_masker
from builtins import FutureWarning
import warnings
labels_path = '/data/simexp/fnadeau/cortex-difumo-693-labels.txt'
cortex_labels = Path(labels_path).read_text().splitlines()
warnings.filterwarnings(action='ignore', category=FutureWarning)
# maps_masker = NiftiMapsMasker(maps_img=cortex_atlas,
#                               mask_img=session.mask_img,
#                               t_r=get_t_r(session.fmri_img),
#                               resampling_target='mask',
#                               **session.masker_defs).fit()
# save_masker(dst=masker_dir, masker=maps_masker, session=session)

trial_type_cols=['trial_type', 'recognition_performance',
                 'ctl_miss_ws_cs']

[session.update(dict(computed_ = get_all_contrasts(fmri_img=session.fmri_img,
                               events=session.events,
                               masker=session.masker,
                               output_type='effect_size',
                               trial_type_cols=trial_type_cols,
                               standardize=True,
                               scale=False,
                               maximize=False,
                               glm_kws=session.glm_defs,
                               design_kws=session.design_defs,
                               feature_labels=cortex_labels)))
 for session in [session00, session01]]

In [ ]:


sub_id, ses_id, task, space = os.path.basename(session.fmri_path).split('_')[:-2]

prefix = '_'.join(os.path.basename(session.fmri_path).split('_')[:-2])

masker_path = sorted(Path(masker_dir).rglob(f'{prefix}*.pickle'))
if masker_path = []:
    masker_path = None,
else: masker_path = masker_path[0]


In [ ]:
# tasks = (session.events.trial_type,
#          session.events.recognition_performance,
#          session.events.iloc[:, -1])
# [t.name for t in tasks]
# (len([p for p in sorted(map(str, sorted(Path(masker_dir).rglob('*.pickle')))) if 'V10' in p]),
#  len([p for p in sorted(map(str, sorted(Path(masker_dir).rglob('*.pickle')))) if 'V03' in p]),)


In [ ]:
session = fetch_fmriprep_session(session=sample(v03+v10, 1)[0])
print(f'{session.sub_id}_{session.ses_id}')

In [ ]:
# di1024 = get_difumo(1024, 3, '/data/simexp/fnadeau/nilearn_atlases/difumo_atlases/')
# foremidbrain_rois = ['thalam', 'pons', 'tegment', 'medulla', 'putamen',
#                      'fornix', 'cerebell', 'pedunc', 'tect', 'diencep',
#                      'striatum', 'fluid', 'colli', 'pars', 'nuc', 'reticu',
#                      'caud', 'ventricle', 'midbrain', 'mamm', 'fossa',
#                      'caps', 'gang', 'globus', 'claus']
# wm_rois = ['fasciculus', 'tract' 'forceps', 'callosum', 'cingulum', 'forceps',
#            'chiasm', 'corticospinal', 'radiation', 'radiata']

# no_cortex = foremidbrain_rois + wm_rois

# not_cortex = di1024.labels.reset_index(drop=False).set_index('difumo_names').T.filter(
#                  regex='|'.join(no_cortex))
# cortex = di1024.labels.reset_index(drop=False).set_index(
#              'difumo_names').T.drop(not_cortex.columns, axis=1).T

# cortex_atlas = nimage.index_img(di1024.maps, cortex.component.tolist())
# cortex_names = cortex.index.tolist()
# print(len(cortex_names))

In [ ]:
# di1024 = get_difumo(1024, 3, '/data/simexp/fnadeau/nilearn_atlases/difumo_atlases/')

dimension, resolution_mm = 1024, 3
# {str(resolution_mm)}
maps_path = sorted(Path(atlases_dir).rglob(f'*{dimension}/{resolution_mm}mm/maps.nii.gz'))[0]
labels_path = sorted(Path(atlases_dir).rglob(f'*labels_{dimension}_dictionary.csv'))[0]

di1024 = Bunch(maps = nibabel.load(maps_path),
               labels = pd.read_csv(labels_path))

foremidbrain_rois = ['thalam', 'pons', 'tegment', 'medulla', 'putamen',
                     'fornix', 'cerebell', 'pedunc', 'tect', 'diencep',
                     'striatum', 'fluid', 'colli', 'pars', 'nuc', 'reticu',
                     'caud', 'ventricle', 'midbrain', 'mamm', 'fossa',
                     'caps', 'gang', 'globus', 'claus']
wm_rois = ['fasciculus', 'tract' 'forceps', 'callosum', 'cingulum', 'forceps',
           'chiasm', 'corticospinal', 'radiation', 'radiata']

no_cortex = foremidbrain_rois + wm_rois

not_cortex = di1024.labels.reset_index(drop=False).set_index('difumo_names').T.filter(
                 regex='|'.join(no_cortex))
cortex = di1024.labels.reset_index(drop=False).set_index(
             'difumo_names').T.drop(not_cortex.columns, axis=1).T

cortex_atlas = nimage.index_img(di1024.maps, cortex.component.tolist())
cortex_names = cortex.index.tolist()

# cortex_rois = difumo_cut_coords.reset_index(drop=False).set_index(
#                   'difumo_names', drop=False).loc[di2.columns].component.values
# maps_path = '/data/simexp/fnadeau/nilearn_atlases/difumo_atlases/1024/3mm/maps.nii.gz'
# new_difumo = nimage.index_img(nimage.load_img(maps_path), cortex_rois)
from cimaq_decoding_utils import get_t_r, get_frame_times, save_masker


trial_type_cols = ['trial_type', 'recognition_performance', 'ctl_miss_ws_cs']


for session in tqdm_(sessions):
    session.update({'masker': NiftiMapsMasker(maps_img=cortex_atlas,
                              mask_img=session.mask_img,
                              resampling_target='mask',
                              t_r=get_t_r(session.fmri_img),
                              **session.masker_defs).fit(),
                    'trial_type_cols': ['trial_type',
                                        'recognition_performance',
                                        'ctl_miss_ws_cs']}),

    dims = f'{str(int(session.masker.maps_img.shape[-1]))}'
    resol = f'{str(int(session.masker.maps_img.header.get_zooms()[0]))}mm'
    masker_name = f'cortex-difumo_{dims}_{resol}_maps-masker.pickle'

    save_masker(dst='/data/simexp/fnadeau/cimaq_maps_maskers/',
                masker_name=masker_name,
                masker=session.masker, session=session)
    
#     session.update({'maps_cdict': get_all_contrasts(fmri_img=session.fmri_img,
#                                         events=session.events,
#                                        masker=session.masker,
#                                        ctl_id='Ctl',
#                                        output_type='effect_size',
#                                        trial_type_cols=trial_type_cols,
#                                        scale=False,
#                                        maximize=False,
#                                        glm_kws=session.glm_defs,
#                                        design_kws=session.design_defs,
#                                        feature_labels=cortex_names)})

In [ ]:
help(json.dump)

In [ ]:
sorted(Path(masker_dir).rglob(f'*{sessions[0].sub_id}*wholebrain*.pickle'))

In [ ]:
session = sess00
timer = timeit.Timer(get_all_contrasts, **dict(fmri_img=session.fmri_img,
                                                     events=session.events,
                                                     masker=maps_masker,
                                                     output_type='effect_size',
                                                     trial_type_cols=['trial_type',
                                                                      'recognition_performance',
                                                                      'ctl_miss_ws_cs'],
                                                   glm_kws=session.glm_defs,
                                                   design_kws=session.design_defs,
                                                   feature_labels=di2.columns.tolist()))

timer.timeit(1)

In [ ]:
ndims = 1024
difumo_cut_coords = pd.read_csv(f'/data/simexp/fnadeau/difumo_{ndims}_dims_3mm_cut_coords.tsv',
                                sep='\t', index_col='component')

di2 = difumo_cut_coords.copy(deep=True).set_index('difumo_names').T
foremidbrain_rois = '|'.join(['thalam', 'pons', 'tegment', 'medulla', 'putamen', 'fornix',
                              'cerebell', 'pedonc', 'tect', 'dien', 'striatum', 'fluid',
                              'colli', 'pars', 'nuc', 'retic', 'caud', 'ventricles',
                              'mamm', 'fossa', 'caps', 'gang', 'globus', 'claus'])
wm_rois = '|'.join(['fasciculus', 'tract' 'forceps', 'callosum', 'cingulum'])
foremidbrain = di2.filter(foremidbrain_rois).columns.tolist()
wm =  di2.filter(wm_rois).columns.tolist()
not_cortex = di2.filter(regex='|'.join([foremidbrain_rois, wm_rois])).columns.tolist()
di2 = di2.drop(not_cortex, axis=1)


spheres_masker = NiftiSpheresMasker(seeds=di2.T[['x', 'y', 'z']].values,
#                                     mask_img=sess00.mask_img,
                                    standardize_confounds=False,
                                    smoothing_fwhm=None,
                                    high_variance_confounds=False,
                                    detrend=False, dtype='f', t_r=2.5,
                                    low_pass=None, high_pass=None,
                                    allow_overlap=True,
                                    standardize=False).fit()


In [ ]:
# ndims = 1024
# difumo_cut_coords = pd.read_csv(f'/data/simexp/fnadeau/difumo_{ndims}_dims_3mm_cut_coords.tsv',
#                                 sep='\t', index_col='component')
# spheres_masker = NiftiSpheresMasker(seeds=difumo_cut_coords[['x', 'y', 'z']].values,
# #                                       mask_img=sess00.mask_img,
#                                       standardize_confounds=False,
#                                       allow_overlap=True,
#                                       standardize=False).fit()


In [ ]:
contrast_dict00 = get_all_contrasts(sess00.fmri_img, sess00.events, output_type='effect_size',
                                    masker=spheres_masker,
                                    trial_type_cols=['trial_type', 'recognition_performance',
                                                     'ctl_miss_ws_cs'],
                                    feature_labels=difumo_cut_coords.difumo_names.tolist(),
                                    design_kws=dict(drift_model=None, hrf_model='spm'),
                                    glm_kws=sess00.glm_defs)


In [ ]:
# import warnings
contrast_dict00.keys()
# sorted(dir(warnings))
# help(warnings.filterwarnings)
# (action='ignore', )

In [ ]:
svc = SVC(tol=0.0001,
          kernel='linear',
          cache_size=5,
          decision_function_shape='ovr',
          class_weight='balanced',
          probability=True)

ovrc = OneVsRestClassifier(svc)

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

X, y = contrast_dict00.signal_matrix, sess00.events.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,
                                                    random_state=42)

# clf = RandomForestClassifier(n_estimators=X_train.shape[0], random_state=42)
ovrc.fit(X_train, y_train)

result = permutation_importance(ovrc, X_train, y_train, n_repeats=2, random_state=42)
perm_sorted_idx = np.argsort(result.importances_mean)

# tree_importance_sorted_idx = np.argsort(ovrc.feature_importances_)
# tree_indices = np.arange(0, len(ovrc.feature_importances_)) + 0.5

print("Accuracy on test data: {:.2f}".format(ovrc.score(X_test, y_test)))



In [ ]:
sorted_importance_labels = tuple(zip(result.importances[perm_sorted_idx].T,
                                     X.iloc[:, perm_sorted_idx].columns))

sorted_importance_labels

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(512, 256))
corr = spearmanr(X).correlation

# Ensure the correlation matrix is symmetric
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)

# We convert the correlation matrix to a distance matrix before performing
# hierarchical clustering using Ward's linkage.
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))

dendro = hierarchy.dendrogram(dist_linkage, labels=X.columns.tolist(),
                              ax=ax1, leaf_rotation=90)
dendro_idx = np.arange(0, len(dendro["ivl"]))

In [ ]:
cluster_ids = hierarchy.fcluster(dist_linkage, 1, criterion="distance")
cluster_id_to_feature_ids = defaultdict(list)

for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

X_train_sel = X_train.iloc[:, selected_features]
X_test_sel = X_test.iloc[:, selected_features]

clf_sel = RandomForestClassifier(n_estimators=X_train_sel.shape[0],
                                 random_state=42)
clf_sel.fit(X_train_sel, y_train)
print(
    "Accuracy on test data with features removed: {:.2f}".format(
        clf_sel.score(X_test_sel, y_test)
    )
)

In [ ]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

# The following import is is required for 3D projection, but not directly used.
# 
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn.datasets import load_iris

help(load_iris)

In [ ]:


np.random.seed(5)

# iris = load_iris(as_frame=True)
# X, y = iris.frame.iloc[:, :-1], iris.frame.iloc[:, -1]
X, y = contrast_dict00.signal_matrix, sess00.events.iloc[:, -1].values

estimators = [
    ("k_means_cimaq_256", KMeans(n_clusters=256)),
    ("k_means_cimaq_64", KMeans(n_clusters=64)),
    ("k_means_iris_bad_init", KMeans(n_clusters=3, n_init=1, init="random")),
]

fignum = 1
titles = ["256 clusters", "64 clusters", "3 clusters, bad initialization"]

#####
bbox_params = dict(alpha=0.2, edgecolor="w", facecolor="w")
#####

for name, est in estimators:
    fig = plt.figure(fignum, figsize=(4, len(estimators)))
    ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
    est.fit(X)
    labels = est.labels_

    ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=labels.astype(float), edgecolor="k")

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel("Petal width")
    ax.set_ylabel("Sepal length")
    ax.set_zlabel("Petal length")
    ax.set_title(titles[fignum - 1])
    ax.dist = 12
    fignum = fignum + 1
    
# Plot the ground truth
fig = plt.figure(fignum, figsize=(4, 3))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)

for name, label in [("Setosa", 0), ("Versicolour", 1), ("Virginica", 2)]:
    ax.text3D(
        X[y == label, 3].mean(),
        X[y == label, 0].mean(),
        X[y == label, 2].mean() + 2,
        name,
        horizontalalignment="center",
        bbox=bbox_params,
    )

# Reorder the labels to have colors matching the cluster results
y = np.choose(y, [1, 2, 0]).astype(float)
ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=y, edgecolor="k")

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel("Petal width")
ax.set_ylabel("Sepal length")
ax.set_zlabel("Petal length")
ax.set_title("Ground Truth")
ax.dist = 12

fig.show()

In [ ]:
ra, pa = 1.0, 0.0

bool(ra and not pa)

In [ ]:
help(neurora)

In [ ]:
import numpy as np
import os
import skimage
import imageio
from pathlib import Path
from PIL import Image
from sklearn.utils import Bunch
from sklearn.feature_extraction.image import extract_patches_2d

impaths = sorted(filter(os.path.isfile, sorted(Path('/data/simexp/fnadeau/wmstim/').iterdir())))
imdict = Bunch(**dict(((os.path.basename(apath), skimage.io.imread(apath, as_gray=True))
               for apath in impaths)))
imgs = list(imdict.values())
from sklearn.preprocessing import StandardScaler

imnames = list(map(os.path.basename, impaths))
images = [np.uint8(skimage.io.imread(apath, as_gray=True).resize(250, 250, Image.ANTIALIAS))
          for apath in impaths]
# patches=[StandardScaler().fit_transform(extract_patches_2d(im, patch_size=(125,125)).flatten().reshape(-1,1))
#          for im in imgs]

In [ ]:

patches = [(patch) for patch in patches]

In [ ]:
from sklearn.metrics import pairwise_distances

patch_vecs = pairwise_distances(np.array(patches))
# sorted(dir(images[0]))
# images[0].shape

In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np


from sklearn import datasets
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.image import extract_patches_2d

# faces = datasets.fetch_olivetti_faces()

# #############################################################################
# Learn the dictionary of images
from PIL import Image

impaths = sorted(filter(os.path.isfile, sorted(Path('/data/simexp/fnadeau/WMStim/').iterdir())))
imdict = dict(((os.path.basename(apath), skimage.io.imread(apath, as_gray=True))
               for apath in impaths))

print("Learning the dictionary... ")

kmeans = MiniBatchKMeans(n_clusters=81, random_state=rng, verbose=True)
patch_size = (20, 20)

buffer = []
t0 = time.time()

# The online learning part: cycle over the whole dataset 6 times
index = 0
for _ in range(6):
    for img in tuple(imdict.values()):
        data = extract_patches_2d(img, patch_size, max_patches=20, random_state=rng)
        data = np.reshape(data, (len(data), -1))
        buffer.append(data)
        index += 1
        if index % 10 == 0:
            data = np.concatenate(buffer, axis=0)
            data -= np.mean(data, axis=0)
            data /= np.std(data, axis=0)
            kmeans.partial_fit(data)
            buffer = []
        if index % 100 == 0:
            print("Partial fit of %4i out of %i" % (index, 6 * len((imdict.values()))))

dt = time.time() - t0
print("done in %.2fs." % dt)

# #############################################################################
# Plot the results
plt.figure(figsize=(4.2, 4))
for i, patch in enumerate(kmeans.cluster_centers_):
    plt.subplot(9, 9, i + 1)
    plt.imshow(patch.reshape(patch_size), cmap=plt.cm.gray, interpolation="nearest")
    plt.xticks(())
    plt.yticks(())


plt.suptitle(
    "Patches of faces\nTrain time %.1fs on %d patches" % (dt, 8 * len((imdict.values()))),
    fontsize=16,
)
plt.subplots_adjust(0.08, 0.02, 0.92, 0.85, 0.08, 0.23)

plt.show()

In [ ]:
# from tqdm import tqdm
# from cimaq_decoding_utils import get_fmri_sessions, fetch_fmriprep_session
# from get_difumo import get_difumo

# difumo = get_difumo(atlases_dir=atlases_dir, n_dims=64, resolution_mm=3)
# difumo_cut_coords = niplot.find_probabilistic_atlas_cut_coords(nimage.load_img(difumo.maps))

# difumo_cut_coords2 = pd.read_csv('/data/simexp/fnadeau/sub-4117134_ses-V03_difumo_1024_dims_3mm_cut_coords.tsv', sep='\t')[['x','y','z']].values
# spheres_masker = NiftiSpheresMasker(seeds=difumo_cut_coords)


# sess00 = fetch_fmriprep_session(session=sample(get_fmri_sessions(topdir=fmriprep_dir,
#                                                                  events_dir=events_dir,
#                                                                  ses_id="V03"), 1)[0])
# sess00.events.duration = sess00.events.duration+sess00.events.isi

# # Extract signals for each trial
# signals = [spheres_masker.fit_transform(img) for img
#            in tqdm(list(nimage.iter_img(sess00.cleaned_fmri)),
#                    desc='Extracting signals')]

# # Create label lists for each classification type
# enc_ctl_labels = get_iso_labels(session=sess00)     
              
# # Match frame times with experimental manip onset
# signals = signals[len(signals)-len(hit_miss_ctl_labels):]

# # Create DataFrame of ROIs (columns) & Signals (rows)
# spacetime_sigs = pd.DataFrame([sig[-1] for sig in signals],
#                               columns=difumo.labels.difumo_names.tolist())

# # Save signal DataFrame
# fname_base = '_'.join([sess00.sub_id, sess00.ses_id,
#                        f'difumo_{difumo.labels.shape[0]}_dims',
#                        f'{str(int(difumo.maps.header.get_zooms()[-2]))}mm'])
# save_dst = f'/data/simexp/fnadeau/{fname_base}.tsv'                     
# spacetime_sigs.to_csv(save_dst, sep='\t', index=True, encoding='UTF-8-SIG')


In [ ]:


# atlases_dir = './nilearn_atlases/difumo_atlases/'
# fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/'
# events_dir = '/data/simexp/fnadeau/CIMAQ_AS_BIDS_4/'

# difumo = get_difumo(atlases_dir=atlases_dir, n_dims=128, resolution_mm=3)
# difumo_cut_coords = niplot.find_probabilistic_atlas_cut_coords(nimage.load_img(difumo.maps))
# spheres_masker = NiftiSpheresMasker(seeds=difumo_cut_coords)


# sess00 = fetch_fmriprep_session(session=sample(get_fmri_sessions(topdir=fmriprep_dir,
#                                                                  events_dir=events_dir,
#                                                                  ses_id="V03"), 1)[0])
# sess00.events.duration = sess00.events.duration+sess00.events.isi

# # Extract signals for each trial
# signals = [spheres_masker.fit_transform(img) for img
#            in tqdm(list(nimage.iter_img(sess00.cleaned_fmri)),
#                    desc='Extracting signals')]
# # Match frame times with experimental manip onset
# signals = signals[len(signals)-len(hit_miss_ctl_labels):]
# # Create DataFrame of ROIs (columns) & Signals (rows)
# spacetime_sigs = pd.DataFrame([sig[-1] for sig in signals],
#                               index=['frame_number'],
#                               columns=difumo.labels.difumo_names.values)
# spacetime_sigs.to_csv(os.path.join(os.getcwd(), 'difumo_1024_dims_3mm_spacetime_signals.tsv'),
#                       sep='\t', index='frame_number', encoding='UTF-8-SIG')

In [ ]:

sess00 = fetch_fmriprep_session(session=sample(get_fmri_sessions(topdir=fmriprep_dir,
                                                                 events_dir=events_dir,
                                                                 ses_id="V03"), 1)[0])
sess00.events.duration = sess00.events.duration+sess00.events.isi

In [ ]:
def u_cond(events):
    evlist = [events.copy(deep=True)]*events.shape[0]
    [item[1].iloc[item[0]].replace(
        {item[1].iloc[item[0]].trial_type:
         'X_'+str(item[1].iloc[item[0]].trial_type)},
     inplace=True)
     for item in enumerate(evlist)]
    return evlist
#     [setattr(evlist[idx].iloc[idx], 'trial_type',
#              'X_'+evlist[idx].iloc[idx].trial_type)
#      for idx in events.index]
#     [setattr(item[1], 'trial_type',
#              ['X_'+trial_row[1].trial_type if
#               trial_row[0]==events.iloc[item[0]].index.tolist()
#               else trial_row[1].trial_type
#               for trial_row in item[1].iterrows()])
#      for item in enumerate(evlist)]
#     return evlist
# test = 
u_cond(sess00.events)[0].trial_type.unique()
# test[0]

In [ ]:
    for row in tqdm(list(events.iterrows()),
                    desc='Computing Beta Map'):
        trial_events = events.copy(deep = True)
        conditions = ['X_'+trial_row[1].trial_type if
                      trial_row[0]==row[0]
                      else trial_row[1].trial_type
                      for trial_row in trial_events.iterrows()]
        trial_events['trial_type'] = conditions

        design = (make_first_level_design_matrix(
                      frame_times=get_frame_times(fmri_img),
                      events=trial_events,
                      **design_kws)
                  if design_kws is not None
                  else make_first_level_design_matrix(
                      frame_times=get_frame_times(fmri_img),
                      events=trial_events))

In [ ]:
# Create label lists for each classification type
from cimaq_decoding_utils import get_iso_labels

enc_ctl_labels = get_iso_labels(session=sess00)

hit_miss_ctl_labels = get_iso_labels(frame_times=sess00.frame_times,
                                     t_r=sess00.t_r,
                                     events=sess00.events.drop('trial_type',
                                                               axis=1).rename(
                                         columns={'recognition_performance':'trial_type'}))

ctl_miss_ws_cs_labels = get_iso_labels(frame_times=sess00.frame_times,
                                       t_r=sess00.t_r,
                                       events=sess00.events.drop('trial_type',
                                                               axis=1).rename(
                                           columns={'ctl_miss_ws_cs':'trial_type'}))

len(enc_ctl_labels),len(hit_miss_ctl_labels),len(ctl_miss_ws_cs_labels)

In [ ]:
# dimension, resolution_mm = 64, 3
# sorted(Path(atlases_dir).rglob(f'*{dimension}/{resolution_mm}mm/*.nii.gz'))
help(NiftiSpheresMasker)

In [ ]:
difumo_cut_coords = pd.read_csv(os.path.join(os.getcwd(),'difumo_64_dims_3mm_cut_coords.tsv'),
                                sep='\t')
difumo_cut_coords.set_axis(['component', 'difumo_names', 'x', 'y', 'z'],
                           axis=1, inplace=True)
spheres_masker = NiftiSpheresMasker(seeds=difumo_cut_coords[['x', 'y', 'z']].values,
                                    standardize='zscore')
# difumo_cut_coords
# sorted(os.listdir())

In [ ]:
help(nilearn.image.smooth_img)

In [ ]:
signals = [spheres_masker.fit_transform(img) for img
           in tqdm(nimage.iter_img(sess00.cleaned_fmri))]

In [ ]:
########################################################################
# Extract signals for each trial
########################################################################
# spheres_masker = NiftiSpheresMasker(seeds=difumo_cut_coords)

signals = [spheres_masker.fit_transform(img) for img
           in tqdm(list(nimage.iter_img(sess00.cleaned_fmri)),
                   desc='Extracting signals')]


########################################################################
# Match frame times with experimental manip onset
########################################################################
signals = signals[len(signals)-len(hit_miss_ctl_labels):]

########################################################################
# Create DataFrame of ROIs (columns) & Signals (rows)
########################################################################
spacetime_sigs = pd.DataFrame([sig[-1] for sig in signals],
                              columns=difumo.labels.difumo_names.values)

########################################################################
# Save Signals DataFrame to csv
########################################################################
spacetime_fname = '_'.join([fname_base, 'seedsignals.tsv'])
spacetime_sigs.to_csv(os.path.join(os.getcwd(), spacetime_fname),
                      sep='\t', index=True, encoding='UTF-8-SIG')


In [ ]:
spacetime_sigs.shape, len(hit_miss_ctl_labels)

In [ ]:
# from cimaq_getbetas2 import sub_tcontrasts1

# betas = sub_tcontrasts1(session=sess00)[0][0]
# sub_mask = sess00.mask_img
# from nilearn.input_data import NiftiLabelsMasker

# sub_label_masker = NiftiLabelsMasker(labels_img=nilearn.image.mean_img(basc),
#                                      standardize=True,
#                                      mask_img=sub_mask,
#                                      memory = 'nilearn_cache', verbose=0)
# sub_label_masker.fit(betas)

In [ ]:
from sklearn.neighbors import BallTree,KDTree,NearestNeighbors
from sklearn import neighbors
from sklearn.metrics import DistanceMetric
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     X_enc_ctl, y_enc_ctl, # x, y
#     test_size=0.3,
#     shuffle=True,
#     stratify=y_enc_ctl,
#     )
# help(PCA)
pca00=PCA().fit(spacetime_sigs)
pca00.X_new_ = pca00.fit_transform(spacetime_sigs)
pca00.covariance = pca00.get_covariance()
pca00.precision = pca00.get_precision()
# pca00.__dict__
pca00.fullfit = pca00._fit_full(spacetime_sigs,0)

# pca_table = pd.DataFrame(pca00.explained_variance_ratio_,
#                          index=spacetime_sigs.columns,
#                          columns=['var_ratio']).sort_values('var_ratio',ascending=False)
# score_samples


In [ ]:
spacetime_zscores = pd.DataFrame(((scipy.stats.zscore(spacetime_sigs[col])
                                   for col in spacetime_sigs.columns))).T

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(
    spacetime_zscores,
    ctl_miss_ws_cs_labels,
    test_size=0.4,
    shuffle=True,
    stratify=ctl_miss_ws_cs_labels)
# Set the parameters by cross-validation
tuned_parameters = [
#     {"kernel": ["rbf"], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000],
#      "class_weight": ["balanced", None]},
    {"kernel": ["linear", "poly", "rbf"], "C": [1, 10, 100, 1000],
     "tol": [0.005, 0.001], "class_weight": ["balanced", None]},    
#     {"kernel": ["linear", "poly"], "C": [0.1, 1],
#      "tol": [0.005, 0.001], "class_weight": ["balanced", None]},
]

scores = ["precision", "recall"]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, scoring="%s_macro" % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found for CIMA-Q:\n")
    print(f'{clf.best_params_}\n')
    print("Grid scores for CIMA-Q:\n")
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:\n")
    print("Model trained on CIMA-Q.")
    print("Scores computed on full evaluation set.\n")
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))

In [ ]:


cv = KFold(n_splits=4)
subSVC = SVC(**{'C': 100, 'class_weight': None, 'kernel': 'poly', 'tol': 0.005})
# 

y_pred = cross_val_predict(subSVC, X_train, y_train,
                           groups=y_train, cv=cv)
# scores
cv_acc = cross_val_score(subSVC, X_train, y_train,
                     groups=y_train, cv=cv)

# evaluate overall model performance on training data
overall_acc = accuracy_score(y_pred=y_pred,
                             y_true=y_train)

overall_cr = classification_report(y_pred=y_pred,
                                   y_true=y_train)

subSVC.fit(X_train, y_train)
y_new = subSVC.predict(X_test)
pred_score = len(list(filter(None, y_new==y_test)))/len(y_test)
resultsA = '\n\n'.join([f'Accuracy - A:\n{overall_acc}',
                        f'Cross-Validation Accuracy - A:\n{cv_acc}',
                        f'Classification Report - A:\n{overall_cr}',
                        f'Spontaneous Prediction Accuracy - A:\n{pred_score}'])
print(resultsA)

In [ ]:
max(pca00.fit(new_spacetime_sigs).explained_variance_ratio_)

In [ ]:
new_spacetime_sigs = pd.DataFrame(pca00.fit_transform(spacetime_sigs),
                                  index=spacetime_sigs.index,
                                  columns=spacetime_sigs.columns)
display(new_spacetime_sigs, spacetime_sigs)

In [ ]:
# components_ (shape=(128, 128) ``n_features by n_features``)
# pca00.explained_variance_.shape,pca00.explained_variance_ratio_.shape
pd.DataFrame(pca00.covariance,index=spacetime_sigs.columns,
             columns=spacetime_sigs.columns)

In [ ]:
# help(pca00._fit)
pca00._fit(spacetime_sigs)

In [ ]:
# sorted(dir(pca00))
# pca00._fit_full(spacetime_sigs,0)[0].shape,pca00.X_new_.shape
# [arr.shape for arr in pca00._fit_full(spacetime_sigs,0)]
pd.DataFrame(pca00._fit_full(spacetime_sigs,0)[-1])

In [ ]:
# Space-Based (ROIs) Normalization
zscored_space_spacetime_sigs = spacetime_sigs.apply(scipy.stats.zscore, raw=True,
                                              axis=1,
                                              result_type='broadcast',
                                              **{'ddof':1})
# Time-Based (Trials) Normalization
zscored_time_spacetime_sigs = spacetime_sigs.apply(scipy.stats.zscore, raw=True,
                                              axis=0,
                                              result_type='broadcast',
                                              **{'ddof':1})
# Subsetting Visual-Occipital Related ROIs
visual_spacetime_sigs = spacetime_sigs.filter(regex='visual|occipital')
visual_zscored_space_spacetime_sigs = zscored_space_spacetime_sigs.filter(regex='visual|occipital')
visual_zscored_time_spacetime_sigs = zscored_time_spacetime_sigs.filter(regex='visual|occipital')


# Save Extracted Signals To ".tsv"
spacetime_fname = '_'.join([fname_base, 'seedsignals.tsv'])
zscored_space_fname = '_'.join([fname_base, 'zscored_space_seedsignals'])
zscored_space_spacetime_sigs.to_csv(f'/data/simexp/fnadeau/{zscored_space_fname}.tsv',
                              sep='\t', index=True, encoding='UTF-8-SIG')

zscored_time_fname = '_'.join([fname_base, 'zscored_time_seedsignals'])


zscored_time_spacetime_sigs.to_csv(f'/data/simexp/fnadeau/{zscored_time_fname}.tsv',
                              sep='\t', index=True, encoding='UTF-8-SIG')

# display(spacetime_sigs, visual_spacetime_sigs)

sigdf = pd.DataFrame(zip(enc_ctl_labels, hit_miss_ctl_labels,
                         ctl_miss_ws_cs_labels),
                     columns=['enc_ctl_labels','hit_miss_ctl_labels',
                              'ctl_miss_ws_cs_labels'])
sigdf['Enc'] = sigdf['enc_ctl_labels']=='Enc'
sigdf['Ctl'] = sigdf['enc_ctl_labels']=='Ctl'
sigdf['Hit'] = sigdf['hit_miss_ctl_labels']=='Hit'
sigdf['Miss'] = sigdf['hit_miss_ctl_labels']=='Miss'
sigdf['Cs'] = sigdf['ctl_miss_ws_cs_labels']=='Cs'
sigdf['Ws'] = sigdf['ctl_miss_ws_cs_labels']=='Ws'
sigdf['Cs'] = sigdf['ctl_miss_ws_cs_labels']=='Cs'
sigdf['Miss'] = sigdf['ctl_miss_ws_cs_labels']=='Miss'

sigdf_name = f'{fname_base}_logistic_regression_labels'
sigdf.to_csv(f'/data/simexp/fnadeau/{sigdf_name}.tsv',
                              sep='\t', index=True, encoding='UTF-8-SIG')
display(sigdf)

In [ ]:

ses_v03 = get_fmri_sessions(topdir=fmriprep_dir,
                            events_dir=events_dir,
                            ses_id="V03")
ses_v10 = get_fmri_sessions(topdir=fmriprep_dir,
                            events_dir=events_dir,
                            ses_id="V10")

actual_ids = list(set([ses.sub_id for ses in ses_v03]+[ses.sub_id for ses in ses_v10]))
actual_ids = [int(itm.split('-')[1]) for itm in actual_ids]

In [ ]:
# t0_path = Path('/data/simexp/fnadeau/dr10_cimaq_archives_propre/dr10_20210901_evaluation_initiale_propre/')
# sorted(t0_path.iterdir())
dr10_path = Path('/data/simexp/fnadeau/dr10_cimaq_archives/')

cogpaths = sorted(filter(os.path.isdir,sorted(Path(str(dr10_path)+'_propre').iterdir())))
t0,t1,t2,t4,t5 = cogpaths[2:-3]
haylpath = cogpaths[-3]
haylfiles = sorted(haylpath.iterdir())
hayl_ids = set(sorted(apath.split('_')[0] for apath
                      in sorted(map(os.path.basename,haylfiles))))

t0_paths = sorted(t0.iterdir())



In [ ]:
import json
with open('/data/simexp/fnadeau/memory_task_headers.json', 'w') as jfile:
    json.dump(memtask_headers,jfile)
    jfile.close()


In [ ]:
'|'.join(list(memtask_headers.keys()))

In [ ]:
dx_clin_36448 = pd.read_csv(t0_paths[11],sep='\t').iloc[:,1:].set_index('pscid')
dx_clin_36448 = dx_clin_36448.loc[[idx for idx in actual_ids
                                   if idx in dx_clin_36448.index]]#.filter(regex='score')
sorted(dx_clin_36448.columns)
# dx_clin_36448

In [ ]:
testnames = sorted(map(os.path.basename,t0_paths))
testnames = list(enumerate(testnames))
testnames
# ['27085_toxicomanie_initial.tsv','32818_habitudes_sommeil_initial.tsv',
#  '32818_habitudes_sommeil_initial.tsv','33165_examen_physique_initial.tsv',]

In [ ]:
toxic_27085 = pd.read_csv(t0_paths[6],sep='\t').iloc[:,1:].set_index('pscid')
toxic_27085 = toxic_27085.loc[[idx for idx in actual_ids
                               if idx in toxic_27085.index]]
sorted(toxic_27085.columns)

In [ ]:
wais_digit_12321 = pd.read_csv(t0_paths[0], sep='\t').iloc[:,1:].set_index('pscid')
todrop = ['data_entry_completion_status','validity',
          'visit_stage','12321_temps_execution',
          '12321_remarques']
wais_digit_12321.drop(todrop,axis=1,inplace=True)
wais_digit_12321 = wais_digit_12321.loc[actual_ids]

moca_12783 = pd.read_csv(t0_paths[1],sep='\t').iloc[:,1:].set_index('pscid').drop(
                 ['data_entry_completion_status', 'candid',
                  'validity','visit_stage'],axis=1).loc[actual_ids]
moca_12783['12783_version'] = moca_12783['12783_version'].str.replace(
                                  'version','').str.replace('francais','fr').str.replace(
                                      'english','en')

flverb_animaux_18057 = pd.read_csv(t0_paths[2],sep='\t').set_index(
                           'pscid').loc[actual_ids].filter(regex='score')

memoria_18087 = pd.read_csv(t0_paths[3],sep='\t').iloc[:,1:].set_index('pscid')
memoria_18087 = memoria_18087.loc[[idx for idx in actual_ids
                                   if idx in memoria_18087.index]].filter(regex='score')

hl_wesch_ri_24918 = pd.read_csv(t0_paths[4],sep='\t').iloc[:,1:].set_index('pscid')
hl_wesch_ri_24918 = hl_wesch_ri_24918.loc[[idx for idx in actual_ids
                                           if idx in hl_wesch_ri_24918.index]].filter(regex='score')

borb_lo_25478 = pd.read_csv(t0_paths[5],sep='\t').iloc[:,1:].set_index('pscid').filter(regex='score')
borb_lo_25478 = borb_lo_25478.loc[[idx for idx in actual_ids
                                   if idx in borb_lo_25478.index]]


somn_epworth_28328 = pd.read_csv(t0_paths[7],sep='\t').iloc[:,1:].set_index('pscid')
somn_epworth_28328 = somn_epworth_28328.loc[[idx for idx in actual_ids
                                             if idx in somn_epworth_28328.index]].filter(regex='score')
somn_epworth_28328 = somn_epworth_28328.iloc[:,:-1]

facename_33288 = pd.read_csv(t0_paths[11],sep='\t').iloc[:,1:].set_index('pscid')
facename_33288 = facename_33288.loc[[idx for idx in actual_ids
                                   if idx in facename_33288.index]].filter(regex='score')

cdr_34013 = pd.read_csv(t0_paths[12],sep='\t').iloc[:,1:].set_index('pscid')
cdr_34013 = cdrsb_34013.loc[[idx for idx in actual_ids
                               if idx in cdr_34013.index]]
cdr_34013 = cdr_34013.iloc[:,-2:]


In [ ]:
cogscores = [wais_digit_12321[['12321_resultat_brut']],moca_12783[['12783_score']],
             flverb_animaux_18057,memoria_18087, hl_wesch_ri_24918,borb_lo_25478,
             cdr_34013, somn_epworth_28328,facename_33288]

In [ ]:
t0_tables = [pd.read_csv(apath,sep='\t').drop(['Unnamed: 0','candid'],
                                              axis=1).set_index('pscid')
                 for apath in sorted(t0.iterdir())]
comments_only = [table.filter(regex='comment|remar|detail')
                 for table in t0_tables]
comment_cols = [table.columns for table in comments_only]
no_comments = [table[0].drop(table[1],axis=1).where(
    table[0].data_entry_completion_status=='complete').where(
    table[0].validity=='valid').dropna(axis=0,how='all')
                 for table in tuple(zip(t0_tables,comment_cols))]
# ['Unnamed: 0' in table.columns.tolist() for table in t0_tables]
t0_bigdata = pd.concat([no_comments[0]]+[table.drop(['candidate_age',
                                                     'validity',
                                                     'visit_stage',
                                                     'data_entry_completion_status'],axis=1)
                                       for table in no_comments[1:]],
                       axis=1).dropna(
                           axis=1,how='all').dropna(
                               axis=0,how='all').drop_duplicates(keep='first')

t0_bigdata = t0_bigdata.replace({'non':int(False),
                                 'oui':int(True)}).drop(['data_entry_completion_status',
                                                         'validity',
                                                         'visit_stage',
                                                         '99498_trouble_personnalite_age'],axis=1)
no_info = [col for col in t0_bigdata if
           t0_bigdata[col].dropna().nunique()<=1]
fifth_missing = no_info = [col for col in t0_bigdata if
                          (t0_bigdata[col].isna().sum() /
                           t0_bigdata.shape[1])*100 >= 20]
t0_bigdata = t0_bigdata.drop(no_info,axis=1)
t0_bigdata


# t0_bigdata.to_csv('/data/simexp/fnadeau/t0_bigdata.tsv',
#                   sep='\t',index='pscid',encoding='UTF-8-SIG')
# t0_bigdata['99498_trouble_personnalite_age'].dropna().unique()
# [dict(tuple((col, table[col].dropna().unique())
#             for col in table.columns))
#  for table in t0_tables]
# display(*tuple(zip(sorted(t0.iterdir()),t0_tables)))
# dict(tuple((col, t0_bigdata[col].dropna().unique())
#            for col in t0_bigdata.columns.tolist()))
# [table.index for table in t0_tables]
# t0_nsub_min = min([table.shape[0] for table in t0_tables])
# 361 rows × 1829 columns
# comments_only
# no_comments

In [ ]:
t0_bigdata.filter(regex='|'.join(list(memtask_headers.keys())))

In [ ]:
sorted(Path('/data/simexp/datasets/cimaq_03-19/derivatives/CIMAQ_fmri_memory/scripts/participant_data').iterdir())

In [ ]:
t0_bigdata.filter(regex='recognition')

In [ ]:
cimaqdir = Path('/data/simexp/datasets/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/')
participants_dir = Path('/data/simexp/datasets/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/participants')
task_dir = Path('/data/simexp/datasets/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files')
import json
memtask_headers = sorted(participants_dir.iterdir())[1]

In [ ]:
import shutil
shutil.copy(src='/data/simexp/datasets/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/participants/MemoTaskParticipantFile_headers.json',
            dst='/data/simexp/fnadeau/MemoTaskParticipantFile_headers.json')

In [ ]:
print(Path('/data/simexp/fnadeau/MemoTaskParticipantFile_headers.json').read_text())

In [ ]:
list(enumerate(table.shape for table in t0_tables))

In [ ]:
cog_test_paths = sorted(filter(os.path.isdir,sorted(Path(str(dr10_path)+'_propre').iterdir())))[2:-2]
cogfiles = sorted(cog_test_paths[0].rglob('*.tsv'))
[cogfiles.extend(list(apath.rglob('*.tsv'))) for apath in cog_test_paths[1:]]
cogfiles = sorted(map(str, cogfiles))
cogfiles

In [ ]:
subdir = 'dr10_20210901_demographie_et_dict_donnees'
init_path = Path(os.path.join(dr10_path,subdir))
init_table_paths = sorted(init_path.rglob('*.tsv'))

init_tables = [pd.read_csv(apath,sep='\t',
                           encoding=get_encoding(apath)).replace(
                               {'donnée_non_disponible':np.nan}).dropna(
                                   axis=1,how='all').dropna(axis=0,how='all')
               for apath in init_table_paths]
display(*init_tables)

In [ ]:


# nsub_min = int(round(max([table.shape[0] for table in init_tables])*0.90, 0))
# ok_tables = [table for table in tuple(zip(init_tables, init_table_paths))
#              if table[0].shape[0]>=nsub_min]

ok_tables = tuple(zip(init_tables,init_table_paths))
newtables=[(itm[1], pd.read_csv(StringIO(unidecode(Path(itm[1]).read_bytes().decode(
                        get_encoding(itm[1]))).lower()),
            sep='\t').replace({'donnee_non_disponible':np.nan}).dropna(
                axis=1,how='all').dropna(axis=0,how='all'))
           for itm in ok_tables]
newdir=f'/data/simexp/fnadeau/dr10_cimaq_archives/{subdir}_propre/'
os.makedirs(newdir,exist_ok=True)
[itm[1].to_csv(os.path.join(newdir,os.path.basename(itm[0])),
               sep='\t',encoding='UTF-8-SIG',index=True)
 for itm in newtables]

In [ ]:
pd.read_table(StringIO(unidecode(ok_tables[0][0].to_string())),sep='\\s+')#.iloc[:,1:].dropna(axis=1,how='all')

In [ ]:
[table[0].to_csv(itm[1],sep='\t',encoding='UTF-8-SIG')]

In [ ]:
# display(*ok_tables)
from io import StringIO
from unidecode import unidecode

def clean_ws(txt:str)->str: return re.sub('    ', '\t', txt)

ok_tables[0].apply(pd.Series.str.lower)
# fmttrs = dict(zip(ok_tables[0].columns.tolist(), [unidecode, str.lower, clean_ws]*ok_tables[0].shape[1]))
# pd.read_fwf(StringIO(str(unidecode(ok_tables[0].to_string().lower())))).iloc[:,1:].dtypes

In [ ]:
demog_path = sorted(dr10_path.iterdir())[4]
demog_dx_init_path = sorted(demog_path.iterdir())[1]
demog_data = pd.read_csv(demog_dx_init_path, sep='\t')
demog_data

In [ ]:
os.chdir('./cimaq_memory/eprime2events/')
from get_encoding import get_encoding
from get_separator import get_separator
os.chdir('/data/simexp/fnadeau/')

In [ ]:
new_hayling_dir = Path('/data/simexp/fnadeau/dr10_20210901_fichiers_hayling_tsv2/')
newdata, badpaths = [], []
for apath in sorted(new_hayling_dir.iterdir()):
    try:
        newdata.append(pd.read_csv(apath, header=0).dropna(axis=1,how='all'))
    except pd.errors.EmptyDataError :
        badpaths.append(apath)


In [ ]:
[table.set_axis(table.iloc[0],axis=1,inplace=True)
 for table in newdata
 if any(['Unnamed: ' in col for col in table.columns])]
        
#         table = table

In [ ]:
for table in newdata:
    if any([col in table.values for col in table.columns]):
        table.reset_index(drop=True, inplace=True)
        table.drop(0,axis=0,inplace=True)
        table.reset_index(drop=True, inplace=True)
        table.dropna(axis=1,how='all',inplace=True)
        

In [ ]:
[itm[0].to_csv(os.path.splitext(itm[1])[0]+'.tsv',
               sep='\t',index=True,header=True,encoding='UTF-8-SIG')
 for itm in tuple(zip(newdata,sorted(new_hayling_dir.iterdir())))]

In [ ]:
old=sorted(Path('/data/simexp/fnadeau/dr10_cimaq_archives/dr10_20210901_fichiers_hayling').iterdir())
new=sorted(Path('/data/simexp/fnadeau/dr10_cimaq_archives/dr10_20210901_fichiers_hayling_tsv2_propre').iterdir())
len(old),len(new)


In [ ]:
# display(*newdata)
table=newdata[4].copy(deep=True)
if any([col in table.values for col in table.columns]):
    table.drop(0,axis=0,inplace=True)
    table.reset_index(drop=True, inplace=True)
table

In [ ]:
hayling_dir = Path('/data/simexp/fnadeau/dr10_cimaq_archives/dr10_20210901_fichiers_hayling')

excels = [p for p in sorted(hayling_dir.iterdir()) if
          os.path.splitext(p)[1] in ('.xlsm','.xlsx')]
hayling_xlsx = sorted(hayling_dir.rglob('*.xlsx'))
hayling_xlsx_tables = []
hayling_xlsm = sorted(hayling_dir.rglob('*.xlsm'))
# hayling_xlsm_tables = []
# for apath in hayling_xlsx:
#     try:
#         hayling_xlsm_tables.append(pd.read_excel(apath,engine='openpyxl'))
#     except (ValueError, AttributeError):
#         print(apath)
#         hayling_xlsx_tables.append(pd.read_xml(apath,
#                                                parser='lxml',
#                                                encoding=get_encoding(apath)))

# [pd.read_excel(apath,engine='xlrd') for apath in hayling_xlsx]

In [ ]:
from unidecode import unidecode
str_exc = lambda exc, l: [i for i in l if all(s not in i for s in exc)]

fpaths = sorted(filter(os.path.isfile, sorted(dr10_path.rglob('*'))))
zipfiles = [str(p) for p in fpaths if str(p).endswith('zip')]

fpaths = str_exc(['.bmp']+zipfiles, [str(p) for p in fpaths])

brokenpaths = [p for p in fpaths if p.endswith('.edat2_2')]
newpaths = [re.sub('.edat2_2','.edat2',p) for p in brokenpaths]
# [os.rename(item[0], item[1]) for item in tuple(zip(brokenpaths, newpaths))]
exts = tuple(set(p.split('.', maxsplit=1)[1] for p in fpaths))
exts
# unicode_paths = [str(Path(unidecode(re.sub('\s{1,}', '_', str(p)))).resolve())
#                  for p in initial_paths]
# [os.rename(item[0], item[1]) for item in tuple(zip(initial_paths, unicode_paths))]

In [ ]:
# ! python3 -m pip install xlrd==1.1.0

In [ ]:
new_csvs = sorted(Path('/data/simexp/fnadeau/dr10_20210901_fichiers_hayling_tsv2/').rglob('*.csv'))
[pd.read_csv(apath) for apath in new_csvs]

In [ ]:


for apath in hayling_xlsx:
    try:
        hayling_xls_tables.append(pd.read_excel(apath,engine='openpyxl'))
    except (ValueError, OSError):
        print(apath)

# newtsvs = [os.path.splitext(p)[0]+'.tsv' for p in excels]
# excel_tables = [pd.read_excel(apath)
#                  for apath in excels]

In [ ]:
# ! python3 -m pip install -U matplotlib

In [ ]:
# set([get_encoding(apath) for apath in hayling_xls])
# ['IBM855','Windows-1252','ISO-8859-1']
buffers = [unidecode(apath.read_bytes().decode('ISO-8859-1'))
           for apath in hayling_xls]

In [ ]:
buffers

In [ ]:
from io import BytesIO

badpath = '/data/simexp/fnadeau/dr10_cimaq_archives/dr10_20210901_fichiers_hayling/8639515_V08_Hayling.xls'
badbuff = BytesIO(Path(badpath).read_bytes().decode('ISO-8859-1').encode('UTF-8'))
get_separator(badpath)
# pd.read_fwf(StringIO(badbuff.read().decode()),engine='python-fwf', sep='\\s+')

In [ ]:
help(pd.DataFrame.to_csv)

In [ ]:
from io import StringIO
lst_exc = lambda exc, l1: [i for i in l1 if i not in exc]
hayling_xls = sorted(hayling_dir.rglob('*.xls'))
xls2tsv = [os.path.splitext(apath)[0]+'.tsv' for apath in hayling_xls]
# hayling_xls = [apath for apath in hayling_xls if apath != badpath]
[pd.read_excel(apath).T.set_index(
    0,drop=True).T.dropna(
        how='all',axis=1).to_csv(os.path.splitext(apath)[0]+'.tsv',
                                 sep='\t', encoding='UTF-8-SIG',
                                 index=True, header=True)
                      for apath in hayling_xls]

# for apath in lst_exc([badpath],hayling_xls):
#     try:
#         hayling_xls_tables.append(pd.read_excel(apath))
#     except ValueError:
#         print(apath)

# set([os.path.splitext(apath)[1] for apath in hayling_xls])
# hayling_xls_tables = [pd.read_excel(apath,
# #                                     encoding='ISO-8859-1',
# #                                     sep=get_separator(apath),
# #                                     header=0,
#                                     engine='openpyxl')
#                       for apath in hayling_xls]
# display(*hayling_xls_tables[:6])
# hayling_files = sorted(map(str,sorted(hayling_dir.iterdir())))[1:]
# hayling_files

# display(*[[pd.read_excel(apath, engine='xlrd', header=0) if
#            os.path.splitext(apath)[1]=='.xls' else
#            pd.read_excel(apath, engine='openpyxl',
#                          header=0)][0]
#           for apath in hayling_files])
# pd.read_excel(os.path.join(hayling_dir,'3002498_V02_Hayling.xlsx'),
#               engine='openpyxl').iloc[:,1:]

In [ ]:
display(*hayling_xls_tables)

In [ ]:
sess00.sub_id
sorted(Path(os.path.join(os.getcwd(),'dr10_20210901_fichiers_irmf_tache/')).iterdir())

In [ ]:
participants_path = '/data/simexp/datasets/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/participants/'
sub_data = pd.read_csv(os.path.join(participants_path,'MemoTaskParticipantFile.tsv'),
                       sep='\t')
sub_data
# list(sub_data.columns)

In [ ]:
# fname_base = '_'.join([sess00.sub_id, sess00.ses_id,
#                        f'difumo_{difumo.labels.shape[0]}_dims',
#                        f'{str(int(difumo.maps.header.get_zooms()[-2]))}mm'])

# spacetime_sigs.to_csv(os.path.join(os.getcwd(), spacetime_fname),
#                       sep='\t', index=True, encoding='UTF-8-SIG')

enc_ctl_table = pd.DataFrame(np.array([enc_ctl_labels]+
                                      [enc_ctl_labels==val
                                       for val in set(enc_ctl_labels.tolist())]).T,
                             columns=['trial_type']+list(set(enc_ctl_labels.tolist())))

hit_miss_ctl_table = pd.DataFrame(np.array([hit_miss_ctl_labels]+
                                           [hit_miss_ctl_labels==val
                                            for val in set(hit_miss_ctl_labels.tolist())]).T,
                                  columns=['trial_type']+list(set(hit_miss_ctl_labels.tolist())))
ctl_miss_ws_cs_table = pd.DataFrame(np.array([ctl_miss_ws_cs_labels]+
                                           [ctl_miss_ws_cs_labels==val
                                            for val in set(ctl_miss_ws_cs_labels.tolist())]).T,
                                  columns=['trial_type']+list(set(ctl_miss_ws_cs_labels.tolist())))


In [ ]:
help(pca00)

In [ ]:
# help(pg.linear_regression)
pg.logistic_regression(X=spacetime_sigs,
                       y=ctl_miss_ws_cs_table.Cs)

In [ ]:
LogisticRegression().fit(X=spacetime_sigs.astype(float),
                       y=ctl_miss_ws_cs_table.iloc[:,1:])

In [ ]:
# help(pd.DataFrame.sparse)
# sorted(dir(pd.arrays))


#### Visual ROIs Subset

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression

# Spatial (ROI-based) & Time (trial-based) z-scores
visual_zspatial_X = visual_zscored_space_spacetime_sigs
visual_ztime_X = visual_zscored_time_spacetime_sigs
visual_logreg_rois = visual_spacetime_sigs.columns

# Using LogisticRegression
# Spatial (ROI-based) zscores

# Control trials
logreg_space_ctl = LogisticRegression()
logreg_space_ctl = logreg_space_ctl.fit(X=visual_zspatial_X,
                                        y=sigdf.Ctl)
logreg_space_ctl_coefs = pd.DataFrame(zip(visual_logreg_rois, logreg_space_ctl.coef_[0]),
                                columns=['ROIs', 'logreg_space_ctl_coefs'])
logreg_space_ctl_coefs = logreg_space_ctl_coefs.sort_values('logreg_space_ctl_coefs',
                                                            ascending=False)

# Encoding trials
logreg_space_enc = LogisticRegression()
logreg_space_enc = logreg_space_enc.fit(X=visual_zspatial_X,
                            y=sigdf.Enc)
logreg_space_enc_coefs = pd.DataFrame(zip(visual_logreg_rois, logreg_space_enc.coef_[0]),
                                columns=['ROIs', 'logreg_space_enc_coefs'])
logreg_space_enc_coefs = logreg_space_enc_coefs.sort_values('logreg_space_enc_coefs',
                                                            ascending=False)

# Time (trial-based) z-scores
# Using LogisticRegression

# Control trials
logreg_time_ctl = LogisticRegression()
logreg_time_ctl = logreg_time_ctl.fit(X=visual_ztime_X,
                            y=sigdf.Ctl)
logreg_time_ctl_coefs = pd.DataFrame(zip(visual_logreg_rois, logreg_time_ctl.coef_[0]),
                                columns=['ROIs', 'logreg_time_ctl_coefs'])
logreg_time_ctl_coefs = logreg_time_ctl_coefs.sort_values('logreg_time_ctl_coefs',
                                                          ascending=False)

# Encoding trials
logreg_time_enc = LogisticRegression()
logreg_time_enc = logreg_time_enc.fit(X=visual_ztime_X,
                            y=sigdf.Enc)
logreg_time_enc_coefs = pd.DataFrame(zip(visual_logreg_rois, logreg_time_enc.coef_[0]),
                                columns=['ROIs', 'logreg_time_enc_coefs'])
logreg_time_enc_coefs = logreg_time_enc_coefs.sort_values('logreg_time_enc_coefs',
                                                          ascending=False)

visual_zscored_logreg_scores = [logreg_space_ctl_coefs,logreg_space_enc_coefs,
                                logreg_time_ctl_coefs, logreg_time_enc_coefs]
[table.sort_values('ROIs', inplace=True) for table in visual_zscored_logreg_scores]
visual_zscored_logreg = pd.concat(([visual_zscored_logreg_scores[0]] +
                                       [table.drop('ROIs',axis=1)
                                        for table in visual_zscored_logreg_scores[1:]]),
                                  axis=1).set_index('ROIs')

visual_zscored_logreg
# pd.DataFrame([row[1].where(row[1]==row[1].max()).dropna()
#               for row in visual_zscored_logreg.iterrows()])
# visual_max_zscored_logreg = visual_zscored_logreg.where(visual_zscored_logreg==visual_zscored_logreg.max())
# visual_min_zscored_logreg = visual_zscored_logreg.where(visual_zscored_logreg==visual_zscored_logreg.min())

# display(visual_max_zscored_logreg,visual_min_zscored_logreg)

In [ ]:
help(LinearRegression().fit)

In [ ]:
# Using LinearRegression
# Spatial (ROI-Based) Z-Scores
# Skipping control trials (would be == Encoding trials*-1)
# Encoding trials
linreg_space_enc = LinearRegression().fit(X=enc_ctl_table.trial_type,
                                          y=visual_zspatial_X)

linreg_space_enc_coefs = pd.DataFrame(zip(visual_spacetime_sigs.columns,
                                          linreg_space_enc.coef_),
                                      columns=['ROIs', 'linereg_space_enc_coefs'])
linreg_space_enc_coefs.sort_values('linereg_space_enc_coefs',ascending=False)
linreg_space_enc_coefs
# # Time (Trial-Based) Z-Scores
# linreg_time_enc = LinearRegression().fit(X=visual_ztime_X,
#                                          y=[float(bool(ylabel)) for ylabel in
#                                              enc_ctl_table.Enc.values.tolist()])
# linreg_time_enc_coefs = pd.DataFrame(zip(visual_spacetime_sigs.columns,
#                                          linreg_time_enc.coef_),
#                                      columns=['ROIs', 'linereg_time_enc_coefs'])
# linreg_time_enc_coefs.linereg_time_enc_coefs = linreg_time_enc_coefs.iloc[:,1].mul(100)
# visual_zscored_linreg_scores = [linreg_space_enc_coefs, linreg_time_enc_coefs]

# [table.sort_values('ROIs', inplace=True)
#  for table in visual_zscored_linreg_scores]

# visual_zscored_linreg = pd.concat([linreg_scores[0]]+[table.drop('ROIs',axis=1)
#                                                       for table in visual_zscored_linreg_scores[1:]],
#                                   axis=1).set_index('ROIs')
# visual_best_zscored_linreg = visual_zscored_linreg.where(visual_zscored_linreg ==
#                                                          visual_zscored_linreg.max())

# pd.concat([visual_best_zscored_logreg,
#            visual_best_zscored_linreg], axis=1).dropna(axis=0, how='all')

#### All 64 DiFuMo ROIs (3mm)

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression

# Spatial (ROI-based) & Time (trial-based) z-scores
zspatial_X = zscored_space_spacetime_sigs
ztime_X = zscored_time_spacetime_sigs
logreg_rois = spacetime_sigs.columns

# Using LogisticRegression
# Spatial (ROI-based) zscores

# Control trials
logreg_space_ctl = LogisticRegression()
logreg_space_ctl = logreg_space_ctl.fit(X=zspatial_X,
                            y=sigdf.Ctl)
logreg_space_ctl_coefs = pd.DataFrame(zip(logreg_rois, logreg_space_ctl.coef_[0]),
                                columns=['ROIs', 'logreg_space_ctl_coefs'])
logreg_space_ctl_coefs = logreg_space_ctl_coefs.sort_values('logreg_space_ctl_coefs',
                                                            ascending=False)

# Encoding trials
logreg_space_enc = LogisticRegression()
logreg_space_enc = logreg_space_enc.fit(X=zspatial_X,
                            y=sigdf.Enc)
logreg_space_enc_coefs = pd.DataFrame(zip(logreg_rois, logreg_space_enc.coef_[0]),
                                columns=['ROIs', 'logreg_space_enc_coefs'])
logreg_space_enc_coefs = logreg_space_enc_coefs.sort_values('logreg_space_enc_coefs',
                                                            ascending=False)

# Time (trial-based) z-scores
# Using LogisticRegression

# Control trials
logreg_time_ctl = LogisticRegression()
logreg_time_ctl = logreg_time_ctl.fit(X=ztime_X,
                            y=sigdf.Ctl)
logreg_time_ctl_coefs = pd.DataFrame(zip(logreg_rois, logreg_time_ctl.coef_[0]),
                                columns=['ROIs', 'logreg_time_ctl_coefs'])
logreg_time_ctl_coefs = logreg_time_ctl_coefs.sort_values('logreg_time_ctl_coefs',
                                                          ascending=False)

# Encoding trials
logreg_time_enc = LogisticRegression()
logreg_time_enc = logreg_time_enc.fit(X=ztime_X,
                            y=sigdf.Enc)
logreg_time_enc_coefs = pd.DataFrame(zip(logreg_rois, logreg_time_enc.coef_[0]),
                                columns=['ROIs', 'logreg_time_enc_coefs'])
logreg_time_enc_coefs = logreg_time_enc_coefs.sort_values('logreg_time_enc_coefs',
                                                          ascending=False)

zscored_logreg_scores = [logreg_space_ctl_coefs,logreg_space_enc_coefs,
                                logreg_time_ctl_coefs, logreg_time_enc_coefs]
[table.sort_values('ROIs', inplace=True) for table in zscored_logreg_scores]
zscored_logreg = pd.concat(([zscored_logreg_scores[0]] +
                                       [table.drop('ROIs',axis=1)
                                        for table in zscored_logreg_scores[1:]]),
                                  axis=1).set_index('ROIs')


pd.DataFrame([row[1].where(row[1]==row[1].max()).dropna()
              for row in zscored_logreg.iterrows()])
best_zscored_logreg = zscored_logreg.where(zscored_logreg ==
                                                         zscored_logreg.max())

# Using LinearRegression
# Spatial (ROI-Based) Z-Scores
# Skipping control trials (would be == Encoding trials*-1)
# Encoding trials
linreg_space_enc = LinearRegression().fit(X=zspatial_X, y=sigdf.Enc)
linreg_space_enc_coefs = pd.DataFrame(zip(spacetime_sigs.columns,
                                          linreg_space_enc.coef_),
                                      columns=['ROIs', 'linereg_space_enc_coefs'])
linreg_space_enc_coefs.sort_values('linereg_space_enc_coefs',ascending=False)

# Time (Trial-Based) Z-Scores
linreg_time_enc = LinearRegression().fit(X=ztime_X, y=sigdf.Enc)
linreg_time_enc_coefs = pd.DataFrame(zip(spacetime_sigs.columns,
                                         linreg_time_enc.coef_),
                                     columns=['ROIs', 'linereg_time_enc_coefs'])
linreg_time_enc_coefs.linereg_time_enc_coefs = linreg_time_enc_coefs.iloc[:,1].mul(100)
zscored_linreg_scores = [linreg_space_enc_coefs, linreg_time_enc_coefs]

[table.sort_values('ROIs', inplace=True)
 for table in linreg_scores]

zscored_linreg = pd.concat([linreg_scores[0]]+[table.drop('ROIs',axis=1)
                                                      for table in linreg_scores[1:]],
                                  axis=1).set_index('ROIs')
best_zscored_linreg = zscored_linreg.where(zscored_linreg ==
                                                         zscored_linreg.max())

pd.concat([best_zscored_logreg,
           best_zscored_linreg], axis=1).dropna(axis=0, how='all')

In [ ]:
## MIST & Niak analysis (St-Laurent, 2019)
# niak_path = Path(os.path.join(os.getcwd(),'niak'))
# mist_csv_paths = list(niak_path.rglob('*.csv'))
# mist_csv_files = [pd.read_csv(apath) for apath in mist_csv_paths]
# mist = datasets.fetch_atlas_basc_multiscale_2015(data_dir=os.path.dirname(atlases_dir),
#                                                  version='asym')
# nilearn.plotting.plot_roi(nimage.load_img(mist.scale444))

## St-Laurent (2019) Contrasts

# from cimaq_getbetas2 import sub_tcontrasts1, sub_tcontrasts2, sub_tcontrasts3
# test1 = sub_tcontrasts1(session=sess00)
# test2 = sub_tcontrasts2(session=sess00)
# test3 = sub_tcontrasts3(session=sess00)
# all_contrasts = list(test1)+list(test2)+list(test3)
# contrast_map = pd.DataFrame(all_contrasts, columns=['images', 'names'])

# label_names = difumo_256.labels.difumo_names.values

# labels_masker_params = dict(labels_img=mean_img(difumo_256.maps),
#                             labels=label_names,
#                             background_label=0,
# #                             mask_img=None,
#                             mask_img=sess00.mask_img,
#                             smoothing_fwhm=None,
#                             standardize=True,
#                             standardize_confounds=False,
#                             high_variance_confounds=False,
#                             detrend=False,
#                             low_pass=None, high_pass=None,
#                             t_r=sess00.fmri_img.header.get_zooms()[-1],
#                             dtype=None,
#                             resampling_target='data', strategy='mean',
#                             reports=True)

# nifti_masker_params = dict(mask_img=sess00.mask_img,
#                             smoothing_fwhm=None,
#                             standardize=False,
#                             standardize_confounds=False,
#                             high_variance_confounds=False,
#                             detrend=False,
#                             low_pass=None, high_pass=None,
#                             t_r=sess00.fmri_img.header.get_zooms()[-1],
#                             dtype=None,
#                             reports=True)

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import OLS

ols_model = OLS(endog=np.array(enc_ctl_labels),
                exog=visual_spacetime_sigs)

In [ ]:
ols_results = ols_model.fit(cov_type='hc3')

In [ ]:
vis_spacetime_enc_ctl = pd.DataFrame(zip(visual_spacetime_sigs.values, enc_ctl_labels),
                                     columns=['signals','trial_type'])
vis_spacetime_enc_ctl

In [ ]:
pg.linear_regression(X=visual_spacetime_sigs.values.tolist(),y=enc_ctl_labels)

In [ ]:
# sigdf.loc[sigdf.labels=='Enc']
sigdf['mean_sigs'] = [np.mean(sig) for sig in sigdf.signals]
sigdf

In [ ]:
pg.logistic_regression(X=sigdf.signals.mean(), y=sigdf.Enc)

In [ ]:
help(pg.logistic_regression)

In [ ]:
pg.logistic_regression(X=sigdf.signals, y=sigdf[['Enc','Ctl']])

In [ ]:
X_B, y_B = sigs, hit_miss_ctl_labels

X_trainB, X_testB, y_trainB, y_testB = train_test_split(
    X_B, y_B, # x, y
    test_size = 0.3,
    shuffle = True, # shuffle dataset before splitting
    stratify = y_B, # keep distribution of conditions consistent betw. train & test sets
    )

In [ ]:
y_pred = cross_val_predict(subSVC, X_train, y_train,
                           groups=y_train, cv=10)
# scores
cv_acc = cross_val_score(subSVC, X_train, y_train,
                     groups=y_train, cv=10)

# evaluate overall model performance on training data
overall_acc = accuracy_score(y_pred=y_pred,
                             y_true=y_train)
overall_cr = classification_report(y_pred=y_pred,
                                   y_true=y_train)

y_predB = cross_val_predict(subSVC, X_trainB, y_trainB,
                            groups=y_train, cv=10)
# scores
cv_accB = cross_val_score(subSVC, X_trainB, y_trainB,
                     groups=y_trainB, cv=10)
print(cv_accB)

# evaluate overall model performance on training data
overall_accB = accuracy_score(y_pred=y_predB,
                             y_true=y_trainB)
overall_crB = classification_report(y_pred=y_predB,
                                    y_true=y_trainB)
resultsA = '\n'.join([f'Accuracy - A:\n{overall_acc}',
                      f'Cross-Validation Accuracy - A\n:{cv_acc}',
                      f'Classification Report - A:\n{overall_cr}'])
resultsB = '\n'.join([f'Accuracy - B:\n{overall_accB}',
                      f'Cross-Validation Accuracy - B\n:{cv_accB}',
                      f'Classification Report - B:\n{overall_crB}'])
print(resultsA,resultsB)


In [ ]:
y_predB = cross_val_predict(subSVC, X_trainB, y_trainB,
                            groups=y_train, cv=10)
# scores
cv_accB = cross_val_score(subSVC, X_trainB, y_trainB,
                     groups=y_trainB, cv=10)
print(cv_accB)

# evaluate overall model performance on training data
overall_accB = accuracy_score(y_pred=y_predB,
                             y_true=y_trainB)
overall_crB = classification_report(y_pred=y_predB,
                                    y_true=y_trainB)

print('Accuracy:',overall_accB)
print(overall_crB)

In [ ]:
subSVC.fit(X_train, y_train)
len(list(filter(None, subSVC.predict(X_test) == y_test)))/len(y_test)

In [ ]:
subSVC.fit(X_trainB, y_trainB)
len(list(filter(None, subSVC.predict(X_testB) == y_testB)))/len(y_testB)

In [ ]:
sess00.keys()

In [ ]:



trial_ends=(sub00.events.onset+abs(sub00.events.onset -
                             sub00.events.offset)+sub00.events.isi).values
trial_intervals=list(itertools.starmap(pd.Interval,
                                       tuple(zip(sub00.events.onset, trial_ends))))
frame_ends=pd.Series(sub00.frame_times)+2.5
frame_intervals=np.array(list(itertools.starmap(pd.Interval,
                                       tuple(zip(sub00.frame_times, frame_ends)))))


sub00.vectors = trial_fmri(sub00.cleaned_fmri, sub00.events)

sub00.sessions=pd.concat(list(pd.DataFrame(
                   itertools.repeat(row[1], len(sub00.vectors.fmri_frames.iloc[row[0]])))
                              for row in sub00.events.iterrows())).reset_index(drop=True)

sub00.sessions_ret = pd.concat(list(pd.DataFrame(
                         itertools.repeat(row[1], len(sub00.vectors.fmri_frames.iloc[row[0]])))
                              for row in sub00.behav.iterrows())).reset_index(drop=True)
# sub00.sessions.set_axis(sub00.events.columns.tolist(), axis=1)
sub00.indexed_fmri = index_img(sub00.cleaned_fmri, sub00.sessions.index)


In [ ]:


# ``load_confounds`` got that covered!

# For decoding, standardizing is often very important
# nifti_masker = NiftiMasker(mask_img=mask_img, sessions=session,
#                            standardize=True, memory='nilearn_cache',
#                            memory_level=1)



#/difumo_atlases/256/3mm/maps.nii.gz'

difumo_256 = get_difumo('../../nilearn_atlases/difumo_atlases/', 256, 3)


# Smaller brain area: faster and quite likely predictive from theory
loc_index = difumo_256.labels.loc[difumo_256.labels.difumo_names.str.match(
                re.compile('lateral occipital', re.I))].index
loc_maps = nilearn.image.index_img(difumo_256.maps, loc_index)

sub00.spheresmasker = NiftiSpheresMasker(tuple(map(tuple,difumo_256.labels[['gm','wm','csf']])),
                                        radius=None, allow_overlap=True,
                                        **sub00.masker_defs)

sub00.mskmapsmasker = NiftiMapsMasker(loc_maps, mask_img=sub00.mask_img,
                                   resampling_target='mask',
                                   allow_overlap=True, **sub00.masker_defs)

sub00.dmapsmasker = NiftiMapsMasker(loc_maps, mask_img=sub00.mask_img,
                                   resampling_target='data',
                                   allow_overlap=True, **sub00.masker_defs)

sub00.regmasker = NiftiMasker(target_affine=sub00.cleaned_fmri.affine,
                              mask_strategy='whole-brain-template',
                              **sub00.masker_defs)

sub00.mskmaps_sig = sub00.mskmapsmasker.fit_transform(sub00.indexed_fmri)
sub00.dmaps_sig = sub00.dmapsmasker.fit_transform(sub00.indexed_fmri)
# fmri_masked = sub00.regmasker.fit_transform(sub00.indexed_fmri)

# from sklearn.feature_selection import f_classif

# f_values, p_values = f_classif(fmri_masked, y)
# p_values = -np.log10(p_values)
# p_values[p_values > 10] = 10
# p_unmasked = get_data(sub00.regmasker.inverse_transform(p_values))

In [ ]:
import nilearn.decoding

from nilearn.image import new_img_like, load_img, get_data

# Cross-validation scheme definition:
    # Here we use a KFold cross-validation on the session.
    # This corresponds to splitting the samples in 4 folds
    # and make 4 runs using each fold as a test
    # set once and the others as learning sets

# Unused yet: groups=stim_categories
stim_categories = sub00.sessions.stim_category.replace(
                      {'K':'kitchen'}).str.lower().fillna('Ctl').values

cv = KFold(n_splits=4)

estimator = LinearSVC(tol=50e-3,
#                       penalty='l2',
#                       loss='hinge',
                      max_iter=1000,
                      class_weight='balanced',
                      fit_intercept=True)

# estimator.fit(sub00.indexed_fmri.get_fdata(), y)
# pp_svc = LinearSVCPure(estimator)
# pp_svc = convert_estimator(SVC())
# lreg = LinearRegression(n_jobs=8).fit(get_data(sub00.indexed_fmri).astype('uint8'), y)
# estimator = LinearClassifierMixinPure(lreg)
y = sub00.sessions.trial_type.values


# The radius is the one of the Searchlight sphere that will scan the volume

###################################################
# Set to 4 since cubic voxels were smoothed to 8mm?


searchlight = nilearn.decoding.SearchLight(mask_img=sub00.dmapsmasker.mask_img,
#                                            sub00.mask_img,
                                           radius=4.0,
                                           estimator=estimator,
                                           n_jobs=10, cv=cv)
searchlight.fit(sub00.indexed_fmri, y)
